- Since time.clock() is no longer supported in python 3.8, clock() has been replaced to perf_counter() in three files of openopt: result.py, runProbSolver.py, ooIter.py
- Solver for linear optimization has changed to gurobi in lpsolver.py 

In [ ]:
from os import makedirs
import pandas as pd
import numpy as np
from freeflux import Model

merged_dict = {
    
    'mmo_mdh' : ['rxn00843_c0', 'rxn14207_c0'], # 2
    'H4MPT_nadh'  : ['rxn07847_c0', 'rxn37236_c0', 'rxn02480_c0', 'rxn02431_c0', 'rxn07849_c0'], # 5
    'H4MPT_nadph' : ['rxn07847_c0', 'rxn07848_c0', 'rxn02480_c0', 'rxn02431_c0', 'rxn07849_c0'], # 5
    'H4F_reductive' : ['rxn01211_c0', 'rxn00907_c0'], # 2
    'ser_to_pg_nadh' : ['rxn00424_c0', 'rxn01011_c0', 'rxn01102_c0'], # 3
    'ser_to_pg_nadph' : ['rxn00424_c0', 'rxn01013_c0', 'rxn01102_c0'], # 3
    'pg_to_pep' : ['rxn01106_c0', 'rxn00459_c0'], # 2
    'mal_to_glx_accoa' : ['rxn00934_c0', 'rxn00331_c0'], # 2
    'EMC_nadh' : ['rxn00178_c0', 'rxn01451_c0', 'rxn02345_c0', 'rxn02167_c0', 'rxn16150_c0', 'rxn16825_c0', 
                  'rxn16151_c0', 'rxn25269_c0', 'rxn03440_c0', 'rxn00682_c0', 'rxn01355_c0', 'rxn01996_c0', 'rxn00602_c0'], # 13
    'EMC_nadph' : ['rxn00178_c0', 'rxn01453_c0', 'rxn02345_c0', 'rxn02167_c0', 'rxn16150_c0', 'rxn16825_c0', 
                  'rxn16151_c0', 'rxn25269_c0', 'rxn03440_c0', 'rxn00682_c0', 'rxn01355_c0', 'rxn01996_c0', 'rxn00602_c0'], # 13
    'cit_to_akg' : ['rxn00974_c0', 'rxn01388_c0', 'rxn00198_c0'], # 3
    'pg_to_gap' : ['rxn01100_c0', 'rxn00781_c0'], # 2
    'f6p_to_gap_dhap' : ['rxn00786_c0', 'rxn00549_c0', 'rxn00747_c0'], # 3
    'x5p_r5p' : ['rxn00777_c0', 'rxn01116_c0'], # 2
    'e4p_dhap_to_s7p' : ['rxn01334_c0', 'rxn01343_c0'] # 2
# total 
}



DILUTION_FROM = [
    'cpd00011u', # co2
    'cpd00023u', # glutamate
    'cpd00024u' # akg
    # 'cpd00041u' # aspartate
    # 'cpd00036u', # succinate
    # 'cpd00072u', # f6p
    # 'cpd00101u', # r5p
    # 'cpd00106u', # fumarate
    # 'cpd00130u', # malate
    # 'cpd00137u' # citrate
    # 'cpd00169u', # phosphoglycerate
    # 'cpd00236u', # e4p
    # 'cpd00238u'  # s7p
]


# estimate fluxes at steady state
def steady_state_fitting():

    ob3b = Model('OB3b_wt')
    ob3b.read_from_file(MODEL_FILE)
    
    with ob3b.fitter('ss') as fit:
        # specify the lableing strategy, 
        # use this method for every labeled substrate
        fit.set_labeling_strategy(
            'cpd01024.ex', 
            labeling_pattern = ['1'], 
            percentage = [0.25], 
            purity = [0.99]
        )
        
        # read measurements
        fit.set_measured_MDVs_from_file(MEASURED_MDVS)
        fit.set_measured_fluxes_from_file(MEASURED_FLUXES)
        
        # set upper and lower bounds for fluxes
        fit.set_flux_bounds('all', bounds = [-100, 100]) 
        
        # solve the fluxes
        fit.prepare(
            dilution_from = DILUTION_FROM, 
            n_jobs = 30
        )
        while True:
            res = fit.solve(solver = 'ralg', max_iters = 1000)
            if res.optimization_successful:
            # if res.opt_objective < 2000:
                break
    
    # save the results
    pd.Series(res.opt_net_fluxes).to_excel(
        OUT_DIR+'/estimated_net_fluxes.xlsx'
    )
    pd.Series(res.opt_total_fluxes).to_excel(
        OUT_DIR+'/estimated_total_fluxes.xlsx'
    )

    net_cis = res.estimate_confidence_intervals(
        which = 'net', 
        confidence_level = 0.95
    )
    total_cis = res.estimate_confidence_intervals(
        which = 'total', 
        confidence_level = 0.95
    )
    pd.DataFrame(net_cis, index = ['LB', 'UB']).T.to_excel(
        OUT_DIR+'/netflux_le_CIs.xlsx'
    )
    pd.DataFrame(total_cis, index = ['LB', 'UB']).T.to_excel(
        OUT_DIR+'/totalflux_le_CIs.xlsx'
    )

    # normal probability plot of residuals
    res.plot_normal_probability(show_fig = False, output_dir = OUT_DIR)
    
    # compare simulations and measurements
    res.plot_simulated_vs_measured_MDVs(show_fig = False, output_dir = OUT_DIR)
    res.plot_simulated_vs_measured_fluxes(show_fig = False, output_dir = OUT_DIR)
    
    # export the contribution matrix
    res.estimate_contribution_matrix(which = 'net').to_excel(
        OUT_DIR+'/netflux_contribMat.xlsx'
    )
    res.estimate_contribution_matrix(which = 'total').to_excel(
        OUT_DIR+'/totalflux_contribMat.xlsx'
    )
    
    # export the sensitivity matrix
    res.estimate_sensitivity(which = 'net').to_excel(
        OUT_DIR+'/netflux_senMat.xlsx'
    )
    res.estimate_sensitivity(which = 'total').to_excel(
        OUT_DIR+'/totalflux_senMat.xlsx'
    )

    df = pd.DataFrame({'opt_resids': res.opt_resids})
    df.to_csv(OUT_DIR + "/opt_resids.csv", index=False)

    sim_flux = pd.Series(res.opt_net_fluxes)
    sim_flux.to_csv(
    OUT_DIR+'/estimated_net_fluxes.csv'
    )
    
    flux = {}
    merged_rxn = list(merged_dict.keys())
    for i in range(len(sim_flux)):
        rxn_id = sim_flux.index[i]
        rxn_flux = sim_flux.values[i]
        
        if rxn_id in merged_rxn:
            rxns = merged_dict[rxn_id]
            for r in rxns:
                if r not in flux:
                    flux[r] = rxn_flux
                else:
                    flux[r] += rxn_flux
        else:
            flux[rxn_id] = rxn_flux
    flux_pd_series = pd.Series(flux)
    flux_pd_series.to_csv(OUT_DIR + 'net_fluxes.csv')
    
    ref = flux['rxn00692_c0']
    norm_flux = pd.Series(
        list(flux.values())/ref, name = 'fluxes', index = list(flux.keys())
    )
    norm_flux.to_csv(OUT_DIR + 'net_normalized_fluxes.csv')

    
    data_list = []
    for key in res.simulated_MDVs.keys():
        for i in range(len(res.simulated_MDVs[key])):
            data_list.append([key, f"M{i}", "Simulated", res.simulated_MDVs[key][i], None])  # No SD for simulated
            data_list.append([key, f"M{i}", "Measured", res.measured_MDVs[key][0][i], res.measured_MDVs[key][1][i]])  # With SD
    
    df = pd.DataFrame(data_list, columns=["Metabolite", "Isotopomer", "Type", "Value", "SD"])
    df.to_csv(OUT_DIR + "/MDVs_data.csv", index=False)
    
    return res


In [ ]:
base_dir         = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM_OB3b/multi_omics/metabolomic/13c_flux_simulation/wt_simulation/'
MODEL_FILE       = base_dir + '0_wt_input_data/WT_10CU_MODEL.xlsx'
MEASURED_MDVS    = base_dir + '0_wt_input_data/WT_10CU_MDV.xlsx'
MEASURED_FLUXES  = base_dir + '0_wt_input_data/WT_10CU_FLUX.xlsx'
OUT_DIR          = base_dir
wt_mfa           = steady_state_fitting()


In [ ]:
base_dir         = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM_OB3b/multi_omics/metabolomic/13c_flux_simulation/mut_1_simulation/'
MODEL_FILE       = base_dir + '0_mut_1_input_data/MUT_1_10CU_MODEL.xlsx'
MEASURED_MDVS    = base_dir + '0_mut_1_input_data/MUT_1_10CU_MDV.xlsx'
MEASURED_FLUXES  = base_dir + '0_mut_1_input_data/MUT_1_10CU_FLUX.xlsx'
OUT_DIR          = base_dir
mut_1_mfa        = steady_state_fitting()


In [ ]:
base_dir         = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM_OB3b/multi_omics/metabolomic/13c_flux_simulation/mut_2_simulation/'
MODEL_FILE       = base_dir + '0_mut_2_input_data/MUT_2_10CU_MODEL.xlsx'
MEASURED_MDVS    = base_dir + '0_mut_2_input_data/MUT_2_10CU_MDV.xlsx'
MEASURED_FLUXES  = base_dir + '0_mut_2_input_data/MUT_2_10CU_FLUX.xlsx'
OUT_DIR          = base_dir
mut_2_mfa        = steady_state_fitting()


In [ ]:
import re
from os import makedirs
import pandas as pd
import freeflux as ff
# import cobra

# cobra_config = cobra.Configuration()
# cobra_config.solver = "gurobi"

# ModelDir = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/Model/'

In [ ]:
md = cobra.io.read_sbml_model(ModelDir + "GEM_ADVE_v13d_Gvalentino.xml")
md.reactions.EX_cpd00007_e0.bounds = (-23,0)
md.reactions.EX_cpd01024_e0.bounds = (-1000,0)
# md.reactions.bio1_biomass.bounds = (0.09,0.11)

for r in md.reactions:
    if r.lower_bound < 0 and r.upper_bound == 0:

        # If the reaction only goes in the reverse direction, flip the reaction direction
        r.lower_bound, r.upper_bound = r.upper_bound, -r.lower_bound
        for met, coeff in r.metabolites.items():
            r.add_metabolites({met: -2 * coeff})


In [ ]:
### Create a dictionary with key = cobra Metabolite, and value = ff Metabolite

met_dict = {}
for met in md.metabolites:
    # met_dict[met.id] = ff.Metabolite(id=met.id)
    met_dict[met] = ff.Metabolite(id=met.id)
print('Cobra Model metabolites num = ', len(md.metabolites))
print('Freeflux Model metabolites num = ', len(met_dict))

In [ ]:
### Create a list of ff Reactions

ff_md = ff.Model('sMMO_ff_model')
rxn_list = []

for rxn in md.reactions:

    if rxn.lower_bound * rxn.upper_bound < 0:
        ff_rxn = ff.Reaction(id=rxn.id, reversible = True)
    else:
        ff_rxn = ff.Reaction(id=rxn.id, reversible = False)


    for met,coeff in rxn.metabolites.items():
        if coeff < 0:
            ff_rxn.add_substrates(substrates = met_dict[met], stoichiometry = abs(coeff))
        elif coeff > 0:
            ff_rxn.add_products(products= met_dict[met], stoichiometry = abs(coeff))

    rxn_list.append(ff_rxn)
    ff_md.add_reactions(reactions=ff_rxn)
    # print(ff_rxn)

### Five metabolites have no reactions:
### cpd00074_c0, cpd00649_c0, cpd00792_c0, cpd11609_c0, cpd11610_c0
print(len(md.metabolites), ff_md.n_metabolites)
print(len(md.reactions), ff_md.n_reactions)

# for met in md.metabolites:
#     if met.id not in ff_md.metabolites:
#         print(met.id)


In [ ]:
with ff_md.optimizer() as opt:

    for rxn in list(ff_md.reactions_info.keys()):
        rxn_lb = md.reactions.get_by_id(rxn).lower_bound
        rxn_ub = md.reactions.get_by_id(rxn).upper_bound
        opt.set_flux_bounds(fluxid = rxn, bounds = [rxn_lb, rxn_ub])
    # opt.prepare()
    res = opt.optimize(objective = {'bio1_biomass': 1}, direction='max')

print('objective:', res.opt_objective)
        

In [ ]:
print('objective:', res.opt_objective)
print(res.opt_fluxes['EX_cpd00007_e0'])
print(res.opt_fluxes['EX_cpd01024_e0'])

In [ ]:
md_sol = md.optimize()

In [ ]:
md_sol.fluxes

In [ ]:
md_sol.fluxes.name

In [ ]:
md.reactions.rxn00251_c0

In [ ]:
import re
from os import makedirs
import pandas as pd
import freeflux as ff
import cobra

cobra_config = cobra.Configuration()
cobra_config.solver = "gurobi"

ModelDir = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/Model/'

In [ ]:
def flip_reaction(rxn_list):
    for r in rxn_list:
        r.lower_bound, r.upper_bound = -r.upper_bound, -r.lower_bound
        for met, coeff in r.metabolites.items():
            r.add_metabolites({met: -2 * coeff})

In [ ]:
md = cobra.io.read_sbml_model(ModelDir + "ff_model.xml")
flip_reaction(
    [md.reactions.rxn00182_c0, 
     md.reactions.rxn00184_c0,
     md.reactions.rxn00248_c0
    ]
)

for r in list(md.reactions):
    if r.lower_bound < 0 and r.upper_bound == 0:
        r.lower_bound, r.upper_bound = r.upper_bound, -r.lower_bound
        for met, coeff in r.metabolites.items():
            r.add_metabolites({met: -2 * coeff})


In [ ]:
## Core metabolic reactoin list

# Methane oxidation to CO2
ch4_oxidation = [
    '00843', '14207', '07847', '07848', '37236', 
    '02480', '02431', '07849', '00371', '00690', 
    '01211', '00907', '12217'
]

central_metabolic_cycles = [
    '00692', '00424', '01011', '01013', '01102', 
    '01106', '00459', '00251', '00248', '00934', 
    '00331', '00990', '00178', '01453', '01451', 
    '02345', '02167', '16150', '16825', '16151', 
    '25269', '03440', '00682', '01355', '01996', 
    '00602', '00256', '00974', '01388', '00198', 
    '08094', '00285', '09272', '00799', '04954'
]

etcetera = [
    '00148', '00154', '00182', '00184', '00085', '00187',
    '00175', '00225', '00173' # AcCoa, Ac cycles
]

glycppp = [
    '01100', '00781', '00786', '00747', '00549',
    '00558', '00777', '01116'
]

cc_rxn_list = ch4_oxidation + central_metabolic_cycles + etcetera + glycppp

for i, rxn in enumerate(cc_rxn_list):
    rxn_id = 'rxn' + rxn + '_c0'
    cc_rxn_list[i] = rxn_id
# cc_rxn_list
md_check_dict = {}
for rxn in cc_rxn_list:
    md_check_dict[rxn] = 1

md_check = pd.Series(md_check_dict, name = 'fluxes')
md_check.to_csv('rxn_check.csv')

# list of non-carbon metabolites that should be remained in model for flux balance
# ATP, O2, NH4, NO3, NADH, NADPH, AcCoA, SucCoA, MalCoA, me/ml/f-thf/h4mpt
required_met = [
    'cpd00002_c0', 'cpd00007_c0', 'cpd00013_c0', 'cpd00209_c0', 'cpd00004_c0','cpd00005_c0'
]
remove_met = [
    'cpd00003_c0', 'cpd00006_c0', # NAD, NADP
    'cpd00008_c0', # ADP
    'cpd00010_c0', # CoA
    'cpd00109_c0', 'cpd00110_c0', # cytochrome red/ox
    'cpd15560_c0', 'cpd15561_c0', # Ubi red/ox
    'cpd00895_c0', 'cpd00087_c0', 'cpd00643_c0' # H4MPT, THF, Mefur
    
]

In [ ]:
# # print(md.reactions.get_by_id('rxn02431_c0').reaction)
# for rxn in cc_rxn_list:
#     r = md.reactions.get_by_id(rxn)
#     if r.lower_bound < 0 and r.upper_bound == 0:            
#         # If the reaction only goes in the reverse direction, flip the reaction direction
#         r.lower_bound, r.upper_bound = r.upper_bound, -r.lower_bound
#         for met, coeff in r.metabolites.items():
#             r.add_metabolites({met: -2 * coeff})
# # print(md.reactions.get_by_id('rxn02431_c0').reaction)

In [ ]:
for rxn in cc_rxn_list:

    # print(md.reactions.get_by_id(rxn).name)
    # print(md.reactions.get_by_id(rxn).id)
    # print(md.reactions.get_by_id(rxn).reaction)

    for met, coeff in md.reactions.get_by_id(rxn).metabolites.items():
        
        # if len(met.formula) >= 10 and met.id not in required_met:
        #     md.reactions.get_by_id(rxn).add_metabolites({met:-coeff})
            
        if 'C' not in met.formula and met.id not in required_met:
            md.reactions.get_by_id(rxn).add_metabolites({met:-coeff})

        elif met.id in remove_met:
            md.reactions.get_by_id(rxn).add_metabolites({met:-coeff})
    # print(md.reactions.get_by_id(rxn).reaction)

In [ ]:
##### create dict for ff reaction dataframe
cc_rxn_dict = {}

### Col_1: reaction_ID
cc_rxn_dict['reaction_ID'] = [md.reactions.get_by_id(rxn).id for rxn in cc_rxn_list]

c_adjust_dict = {'-coa': 21, 
                 'h4mpt': 30,
                 'tetrahydromethanopterin': 30,
                 'methanofuran': 34,
                 'tetrahydrofolate': 19}
### Col_2: substrate_IDs(atom)
### Col_3: product_IDs(atom)
carbon_label = 'abcdefghijklmnopqrstuvwxyz'
# one_carbon_list = ['methanopterin', 'h4mpt', 'methanofuran', 'tetrahydrofolate']
# two_carbon_list = ['acetyl-coa']
# four_carbon_list = ['acetoacetyl-coa', 'hydroxybutanoyl-coa', 'hydroxybutyryl-coA', 'crotonyl-coa']
# five_carbon_list = ['ethylmalonyl-coa', 'methylsuccinyl-coa', 'mesaconyl-coa', 'methylmalyl-coa',
#                     '']
subs_list = []
prod_list = []


for rxn in cc_rxn_list:
   
    rxn_subs = []
    rxn_prod = []

    carbon_label_subs = 'abcdefghijklmnopqrstuvwxyz'
    carbon_label_prod = 'abcdefghijklmnopqrstuvwxyz'
    
    for met, coeff in md.reactions.get_by_id(rxn).metabolites.items():
      
        met_formula = met.formula
        
        met_unpack = [s for s in met_formula]
        met_re = re.split('(\d+)',met_formula)
        
        try:
            c = met_unpack.index('C') + 1
        except:
            c = 0
       
        if c != 0: 
            try:
                c_num_ind = met_re.index('C')+1
                c_num = int(met_re[c_num_ind])
            except:
                c_num = 1
        else:
            c_num = 0 
        
        if '_c0' in met.name:
            c_check = met.name[:-3]
        elif '_e0' in met.name:
            c_check = met.name[:-3]
        else:
            c_check = met.name
        
        for c in list(c_adjust_dict.keys()):
            if c in c_check.lower():
                c_num = c_num - c_adjust_dict[c]
                
        if c_num > 8:
            c_num = 0            
        
        
        if abs(coeff) != 1:
            met_id = str(abs(coeff))+met.id[0:-3]
        else:
            met_id = met.id[0:-3]

        
        
        if c_num != 0:
            if coeff < 0:
                rxn_subs.append(met_id+'('+carbon_label_subs[:c_num]+')')
                carbon_label_subs = carbon_label_subs[c_num:]
            elif coeff > 0:
                rxn_prod.append(met_id+'('+carbon_label_prod[:c_num]+')')
                carbon_label_prod = carbon_label_prod[c_num:]
        else:
            if coeff < 0:
                rxn_subs.append(met_id)
            elif coeff > 0:
                rxn_prod.append(met_id)        
        
        # print(rxn, c_check, c_num, met_id)
        # print(rxn_subs)
        # print(rxn_prod)
    
    subs_list.append('+'.join(rxn_subs))
    prod_list.append('+'.join(rxn_prod))

cc_rxn_dict['substrate_IDs(atom)'] = subs_list
cc_rxn_dict['product_IDs(atom)'] = prod_list


### Col_4: reversibility
reversibility = []
for rxn in cc_rxn_list:

    lb = md.reactions.get_by_id(rxn).lower_bound
    ub = md.reactions.get_by_id(rxn).upper_bound

    if lb >= 0 and ub > 0 :
        reversibility.append('0')
    elif lb < 0 and ub > 0 :
        reversibility.append('1')
    elif lb < 0 and ub <= 0 :
        reversibility.append('0')

cc_rxn_dict['reversibility'] = reversibility        

cc_rxn_df = pd.DataFrame(cc_rxn_dict)
# cc_rxn_df.to_csv('cc_rxn_df.csv')

In [ ]:
aa_rxn_dict = {}
## Check if cpd00032 (cpd00032) required for biomass
## SO4 (cpd00048) exchange required

aa_rxn_dict['reaction_ID'] = [
# 'gdh',
# 'gs',
'pro',
'arg',
'asp',
'asn',
'ala',
'ser',
# 'gly1',
# 'gldc',
# 'gly2',
'cys',
'lys1',
'lys2',
'thr',
'met',
'val',
'leu',
'ile',
'phe',
'tyr',
'trp',
'his',

'biom'
]

aa_rxn_dict['substrate_IDs(atom)'] = [
# 'cpd00024(abcde)+cpd00005+cpd00013'
# 'cpd00023(abcde)+cpd00002+cpd00013'
'cpd00023(abcde)+cpd00002+2cpd00005',
'cpd00023(abcde)+cpd00011(f)+cpd00053(ghijk)+cpd00041(lmno)+cpd00022(pq)+5cpd00002+cpd00005',
'cpd00032(abcd)+cpd00023(efghi)',
'cpd00041(abcd)+2cpd00002+cpd00013',
'cpd00020(abc)+cpd00004+cpd00013',
'cpd00169(abc)+cpd00023(defgh)',
# 'cpd00054(abc)'
# 'cpd00033(ab)'
# 'cpd00161(abcd)'
'cpd00054(abc)+cpd00022(de)+3cpd00002+4cpd00005+cpd00048',
'cpd00041(abcd)+cpd00020(efg)+cpd00023(hijkl)+cpd00078(mnop)+cpd00002+2cpd00005',
'cpd00516(abcdefg)',
'cpd00041(abcd)+2cpd00002+2cpd00005',
'cpd00041(abcd)+cpd00345(e)+cpd00084(fgh)+cpd00078(ijkl)+cpd00002+2cpd00005',
'cpd00020(abc)+cpd00020(def)+cpd00023(ghijk)+cpd00005',
'cpd00022(ab)+cpd00020(cde)+cpd00020(fgh)+cpd00023(ijklm)+cpd00005',
'cpd00161(abcd)+cpd00020(efg)+cpd00023(hijkl)+cpd00005',
'cpd00061(abc)+cpd00061(def)+cpd00236(ghij)+cpd00023(klmno)+cpd00002+cpd00005',
'cpd00061(abc)+cpd00061(def)+cpd00236(ghij)+cpd00023(klmno)+cpd00002+cpd00005',
'cpd00054(abc)+cpd00101(defgh)+cpd00061(ijk)+cpd00236(lmno)+cpd00061(pqr)+cpd00053(stuvw)+3cpd00002+cpd00005',
'cpd00101(abcde)+cpd00201(f)+cpd00053(ghijk)+cpd00041(lmno)+5cpd00002',

'0.488cpd00035+0.281cpd00051+0.229cpd00132+0.229cpd00041+0.087cpd00084+0.250cpd00023+0.250cpd00053+0.582cpd00033+0.090cpd00119+0.276cpd00322+0.428cpd00107+0.326cpd00039+0.146cpd00060+0.176cpd00066+0.210cpd00129+0.205cpd00054+0.241cpd00161+0.054cpd00065+0.131cpd00069+0.402cpd00156+0.205cpd00079+0.071cpd00072+0.754cpd00101+0.129cpd00102+0.619cpd00169+0.051cpd00061+0.083cpd00020+2.510cpd00022+0.087cpd00024+0.340cpd00032+0.443cpd00125+55cpd00002+5.363cpd00005'
]

aa_rxn_dict['product_IDs(atom)'] = [
# 'cpd00023(abcde)',
# 'cpd00053(abcde)',
'cpd00129(abcde)',
'cpd00051(abcdef)+cpd00024(ghijk)+cpd00106(lmno)+cpd00029(pq)',
'cpd00041(abcd)+cpd00024(efghi)',
'cpd00132(abcd)',
'cpd00035(abc)',
'cpd00054(abc)+cpd00024(defgh)+cpd00004',
# 'cpd00033(ab)+cpd00125(c)',
# 'cpd00011(a)+cpd00125(b)+cpd00004+cpd00013',
# 'cpd00033(ab)+cpd00022(cd)+cpd00004',
'cpd00084(abc)+cpd00029(de)',
'cpd00516(abcdgfe)+cpd00024(hijkl)+cpd00036(mnop,ponm)',
'cpd00039(abcdef)+cpd00011(g)',
'cpd00161(abcd)',
'cpd00060(abcde)+cpd00020(fgh)+cpd00036(ijkl,lkji)+cpd00013',
'cpd00156(abcef)+cpd00011(d)+cpd00024(ghijk)',
'cpd00107(abdghe)+cpd00011(c)+cpd00011(f)+cpd00024(ijklm)+cpd00004',
'cpd00322(abfcdg)+cpd00011(e)+cpd00024(hijkl)+cpd00013',
'cpd00066(abcefghij)+cpd00011(d)+cpd00024(klmno)',
'cpd00069(abcefghij)+cpd00011(d)+cpd00024(klmno)+cpd00004',
'cpd00065(abcedklmnoj)+cpd00011(i)+cpd00102(fgh)+cpd00020(pqr)+cpd00023(stuvw)',
'cpd00119(edcbaf)+cpd00024(ghijk)+cpd00106(lmno)+2cpd00004',

'39.68Biomass+1.455NADH'
]

aa_rxn_dict['reversibility'] = [
# '0',
# '0',
'0',
'0',
'0',
'0',
'0',
'0',
# '1',
# '1',
# '0',
'0',
'0',
'0',
'0',
'0',
'0',
'0',
'0',
'0',
'0',
'0',
'0',
'0'
]

aa_rxn_df = pd.DataFrame(aa_rxn_dict)
# aa_rxn_df
# aa_rxn_df.to_csv('aa_rxn_df.csv')

In [ ]:
ex_rxn_dict = {}

ex_rxn_dict['reaction_ID'] = [
'atps1',
'atps2',
'nnt',
'atpm',
'ch4in',
'co2out',	
'o2in',	
# 'acout',	
'nh4in',	
'so4in'
]

ex_rxn_dict['substrate_IDs(atom)'] = [
'cpd00004+0.5cpd00007',
'cpd00982+0.5cpd00007',
'cpd00004',
'cpd00002',
'cpd01024.ex(a)',
'cpd00011(a)',
'cpd00007.ex',
# 'Ac(ab)',
'cpd00209.ex+cpd00005',
'cpd00048.ex'
]

ex_rxn_dict['product_IDs(atom)'] = [
'2cpd00002',
'cpd00002',
'cpd00005',
'cpd00002.ex',
'cpd01024(a)',
'cpd00011.ex(a)',
'cpd00007',
# 'Ac.ex(ab)',
'cpd00013',
'cpd00048'
]

ex_rxn_dict['reversibility'] = [
'0',
'0',
'0',
'0',
'0',
'0',
'0',
# '0',
'0',
'0'
]

ex_rxn_df = pd.DataFrame(ex_rxn_dict)
    
	

In [ ]:
tk_ta_rxn_dict = {}

tk_ta_rxn_dict['reaction_ID'] = [
'tkt1',
'tkt2',	
'tkt3',	
'tal1',	
'tal2'	
]

tk_ta_rxn_dict['substrate_IDs(atom)'] = [
'cpd00198(abcde)',	
'cpd00072(abcdef)',	
'cpd00238(abcdefg)',	
'cpd00072(abcdef)',	
'cpd00238(abcdefg)'	
]

tk_ta_rxn_dict['product_IDs(atom)'] = [
'TK(ab)+cpd00102(cde)',
'TK(ab)+cpd00236(cdef)',
'TK(ab)+cpd00101(cdefg)',
'TA(abc)+cpd00102(def)',
'TA(abc)+cpd00236(defg)'
]

tk_ta_rxn_dict['reversibility'] = [
'1',
'1',
'1',
'1',
'1'
]

tk_ta_rxn_df = pd.DataFrame(tk_ta_rxn_dict)


In [ ]:
ff_rxn_df = pd.concat(
    [cc_rxn_df, aa_rxn_df, ex_rxn_df, tk_ta_rxn_df], 
    axis=0,ignore_index = True
)

ff_rxn_df
ff_rxn_df.to_csv('ff_rxn_df.csv')

## Fix rxn09272: add FADH (cpd00982) in products
## Fix rxn01355: change to carbonate (cpd00242) to carbon dioxide (cpd00011)
## Fix rxn00085: change carbon label at substrate cpd00053(fghij) to (abcde)

## rxn00178: substrate cpd00279(abcd) changed to cpd00279(abab)

## Delete: 
# 'nnt'
# 'rxm00990_c0'

In [ ]:
# Test FBA with ff_rxn_df
base_dir         = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/Fit_3/'
MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL_TEST.xlsx'

ff_md = ff.Model('ff_model_test')
ff_md.read_from_file(MODEL_FILE)
with ff_md.optimizer() as opt:
    opt.set_flux_bounds('all', bounds = [-100, 100])
    # opt.set_flux_bounds('ch4in', bounds = [13.5, 15])
    opt.set_flux_bounds('o2in', bounds = [21.2, 21.2])
    opt.set_flux_bounds('atpm', bounds = [4.21, 4.21])
    # opt.set_flux_bounds('pyrin', bounds = [0, 2])

    # opt.set_flux_bounds('o2in', bounds = [23, 23])

    # opt.set_flux_bounds('rxn07847_c0', bounds = [0, 100])
    # opt.set_flux_bounds('rxn07848_c0', bounds = [0, 100])
    # opt.set_flux_bounds('rxn37236_c0', bounds = [0, 100])
    # opt.set_flux_bounds('rxn02480_c0', bounds = [0, 100])
    # opt.set_flux_bounds('rxn02431_c0', bounds = [0, 100])
    # opt.set_flux_bounds('rxn07849_c0', bounds = [0, 100])
    # opt.set_flux_bounds('rxn00371_c0', bounds = [0, 100])
    # opt.set_flux_bounds('rxn00248_c0', bounds = [-100, 0])

    # opt.set_flux_bounds('rxn00182_c0', bounds = [0, 0])
    # opt.set_flux_bounds('rxn00184_c0', bounds = [0, 0])
    # opt.set_flux_bounds('atps2', bounds = [0, 1])
    # opt.set_flux_bounds('biom', bounds = [0.10, 0.10])
    
    opt.prepare()
    res = opt.optimize(objective = {'biom': 1})
    # res = opt.optimize(objective = {'o2in': 1})

fluxes_pd_series = pd.Series(
    [res.opt_fluxes[rxn] for rxn in ff_md.reactions], name = 'fluxes', index = ff_md.reactions
)
fluxes_pd_series.to_excel(base_dir + '0_data/WT_FBA_ref_TEST.xlsx')

# print('objective:  ',res.opt_objective)
# print('growth:     ',res.opt_fluxes['biom'])
# print('methane:    ',res.opt_fluxes['ch4in'])
# print('oxygen:     ',res.opt_fluxes['o2in'])
# print('atpase_1:   ',res.opt_fluxes['atps1'])
# print('atpase_2:   ',res.opt_fluxes['atps2'])
# print('acetate_out:',res.opt_fluxes['acout'])

# print('pyruvate_in:',res.opt_fluxes['pyrin'])
# print('citrate_out:',res.opt_fluxes['citout'])
# print('succ_out:   ',res.opt_fluxes['succout'])
# print('fum_out:    ',res.opt_fluxes['fumout'])

# print('formate_out:',res.opt_fluxes['forout'])



In [ ]:
import os

base_dir         = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/Fit_1/'
MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL_TEST.xlsx'
MEASURED_MDVS    = base_dir + '0_data/WT_10CU_MDV_TEST.xlsx'
MEASURED_FLUXES  = base_dir + '0_data/WT_10CU_FLUX.xlsx'
INI_FLUX         = base_dir + '0_data/WT_FBA_ref_TEST.xlsx'

results_fit_1 = []
obj_fit_1     = []
for i in range(11, 21):

    # make directory
    result_dir = base_dir + str(i)
    os.mkdir(result_dir)
    
    model = ff.Model('wt_10cu')
    model.read_from_file(MODEL_FILE)
    
    with model.fitter('ss') as fit:
        
        # specify the lableing strategy, use this method for every labeled substrate
        fit.set_labeling_strategy(
            'cpd01024.ex', 
            labeling_pattern = ['0', '1'], 
            percentage = [0.75, 0.25], 
            purity = [1, 1]
        )

        # set bounds for fluxes
        fit.set_flux_bounds('all', bounds = [-50, 50])
        fit.set_flux_bounds('o2in', bounds = [10, 30])
        fit.set_flux_bounds('ch4in', bounds = [10, 30])
        fit.set_flux_bounds('atpm', bounds = [20, 30])
        fit.set_flux_bounds('biom', bounds = [0.04, 0.2])

        # read measurements
        fit.set_measured_MDVs_from_file(MEASURED_MDVS)
        fit.set_measured_fluxes_from_file(MEASURED_FLUXES)

        # slove the fluxes using initial flux guess by FBA
        # fit.prepare(n_jobs = 1)
        # res = fit.solve(
        #     solver = 'slsqp',
        #     ini_fluxes = INI_FLUX,
        #     fit_measured_fluxes = True,
        # )
        
        # slove the fluxes without reference
        fit.prepare(n_jobs = 1)
        res = fit.solve(
            solver = 'slsqp'
        )
        results_fit_1.append(res)
        obj_fit_1.append(res.opt_objective)

    
    pd.Series(res.opt_net_fluxes).to_csv(
        base_dir + '/estimated_net_fluxes_' + str(i) + '.csv'
    )
    pd.Series(res.opt_total_fluxes).to_csv(
        result_dir + '/estimated_total_fluxes_' + str(i) + '.csv'
    )
    
    # normal probability plot of residuals
    res.plot_normal_probability(show_fig = False, output_dir = result_dir)
    
    # compare simulations and measurements
    res.plot_simulated_vs_measured_MDVs(show_fig = False, output_dir = result_dir)
    res.plot_simulated_vs_measured_fluxes(show_fig = False, output_dir = result_dir)
    
    # export the contribution matrix
    res.estimate_contribution_matrix(which = 'net').to_csv(
        result_dir + '/netflux_contribMat_' + str(i) + '.csv'
    )
    res.estimate_contribution_matrix(which = 'total').to_csv(
        result_dir + '/totalflux_contribMat_' + str(i) + '.csv'
    )
    
    # export the sensitivity matrix
    res.estimate_sensitivity(which = 'net').to_csv(
        result_dir + '/netflux_senMat_' + str(i) + '.csv'
    )
    res.estimate_sensitivity(which = 'total').to_csv(
        result_dir + '/totalflux_senMat_' + str(i) + '.csv'
    )






In [ ]:
import os

base_dir         = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/Fit_2/'
MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL_TEST.xlsx'
MEASURED_MDVS    = base_dir + '0_data/WT_10CU_MDV_TEST.xlsx'
MEASURED_FLUXES  = base_dir + '0_data/WT_10CU_FLUX.xlsx'
INI_FLUX         = base_dir + '0_data/WT_FBA_ref_TEST.xlsx'

results_fit_2 = []
obj_fit_2     = []
for i in range(1, 11):

    # make directory
    result_dir = base_dir + str(i)
    os.mkdir(result_dir)
    
    model = ff.Model('wt_10cu')
    model.read_from_file(MODEL_FILE)
    
    with model.fitter('ss') as fit:
        
        # specify the lableing strategy, use this method for every labeled substrate
        fit.set_labeling_strategy(
            'cpd01024.ex', 
            labeling_pattern = ['0', '1'], 
            percentage = [0.75, 0.25], 
            purity = [1, 1]
        )

        # set bounds for fluxes
        fit.set_flux_bounds('all', bounds = [-50, 50])
        fit.set_flux_bounds('o2in', bounds = [10, 30])
        fit.set_flux_bounds('ch4in', bounds = [10, 30])
        fit.set_flux_bounds('atpm', bounds = [20, 30])
        fit.set_flux_bounds('biom', bounds = [0.04, 0.2])

        # read measurements
        fit.set_measured_MDVs_from_file(MEASURED_MDVS)
        fit.set_measured_fluxes_from_file(MEASURED_FLUXES)

        # slove the fluxes using initial flux guess by FBA
        # fit.prepare(n_jobs = 1)
        # res = fit.solve(
        #     solver = 'slsqp',
        #     ini_fluxes = INI_FLUX,
        #     fit_measured_fluxes = True,
        # )
        
        # slove the fluxes without reference
        fit.prepare(n_jobs = 1)
        res = fit.solve(
            solver = 'slsqp'
        )
        results_fit_2.append(res)
        obj_fit_2.append(res.opt_objective)

    
    pd.Series(res.opt_net_fluxes).to_csv(
        base_dir + '/estimated_net_fluxes_' + str(i) + '.csv'
    )
    pd.Series(res.opt_total_fluxes).to_csv(
        result_dir + '/estimated_total_fluxes_' + str(i) + '.csv'
    )
    
    # normal probability plot of residuals
    res.plot_normal_probability(show_fig = False, output_dir = result_dir)
    
    # compare simulations and measurements
    res.plot_simulated_vs_measured_MDVs(show_fig = False, output_dir = result_dir)
    res.plot_simulated_vs_measured_fluxes(show_fig = False, output_dir = result_dir)
    
    # export the contribution matrix
    res.estimate_contribution_matrix(which = 'net').to_csv(
        result_dir + '/netflux_contribMat_' + str(i) + '.csv'
    )
    res.estimate_contribution_matrix(which = 'total').to_csv(
        result_dir + '/totalflux_contribMat_' + str(i) + '.csv'
    )
    
    # export the sensitivity matrix
    res.estimate_sensitivity(which = 'net').to_csv(
        result_dir + '/netflux_senMat_' + str(i) + '.csv'
    )
    res.estimate_sensitivity(which = 'total').to_csv(
        result_dir + '/totalflux_senMat_' + str(i) + '.csv'
    )






In [ ]:
obj_fit_1

In [ ]:
obj_fit_2

In [ ]:
import os

base_dir         = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/Fit_1/'
MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL_TEST.xlsx'
MEASURED_MDVS    = base_dir + '0_data/WT_10CU_MDV_TEST.xlsx'
MEASURED_FLUXES  = base_dir + '0_data/WT_10CU_FLUX.xlsx'
INI_FLUX         = base_dir + '0_data/WT_FBA_ref_TEST.xlsx'

results_fit_ini_1 = []
obj_fit_ini_1     = []
for i in range(21, 31):

    # make directory
    result_dir = base_dir + str(i)
    os.mkdir(result_dir)
    
    model = ff.Model('wt_10cu')
    model.read_from_file(MODEL_FILE)
    
    with model.fitter('ss') as fit:
        
        # specify the lableing strategy, use this method for every labeled substrate
        fit.set_labeling_strategy(
            'cpd01024.ex', 
            labeling_pattern = ['0', '1'], 
            percentage = [0.75, 0.25], 
            purity = [1, 1]
        )

        # set bounds for fluxes
        fit.set_flux_bounds('all', bounds = [-50, 50])
        fit.set_flux_bounds('o2in', bounds = [10, 30])
        fit.set_flux_bounds('ch4in', bounds = [10, 30])
        fit.set_flux_bounds('atpm', bounds = [20, 30])
        fit.set_flux_bounds('biom', bounds = [0.04, 0.2])

        # read measurements
        fit.set_measured_MDVs_from_file(MEASURED_MDVS)
        fit.set_measured_fluxes_from_file(MEASURED_FLUXES)


        
        # slove the fluxes using initial flux guess by FBA
        fit.prepare(n_jobs = 1)
        res = fit.solve(
            solver = 'slsqp',
            ini_fluxes = INI_FLUX,
            fit_measured_fluxes = True,
        )
        
        # # slove the fluxes without reference
        # fit.prepare(n_jobs = 1)
        # res = fit.solve(
        #     solver = 'slsqp'
        # )


        
        results_fit_ini_1.append(res)
        obj_fit_ini_1.append(res.opt_objective)

    
    pd.Series(res.opt_net_fluxes).to_csv(
        base_dir + '/estimated_net_fluxes_' + str(i) + '.csv'
    )
    pd.Series(res.opt_total_fluxes).to_csv(
        result_dir + '/estimated_total_fluxes_' + str(i) + '.csv'
    )
    
    # normal probability plot of residuals
    res.plot_normal_probability(show_fig = False, output_dir = result_dir)
    
    # compare simulations and measurements
    res.plot_simulated_vs_measured_MDVs(show_fig = False, output_dir = result_dir)
    res.plot_simulated_vs_measured_fluxes(show_fig = False, output_dir = result_dir)
    
    # export the contribution matrix
    res.estimate_contribution_matrix(which = 'net').to_csv(
        result_dir + '/netflux_contribMat_' + str(i) + '.csv'
    )
    res.estimate_contribution_matrix(which = 'total').to_csv(
        result_dir + '/totalflux_contribMat_' + str(i) + '.csv'
    )
    
    # export the sensitivity matrix
    res.estimate_sensitivity(which = 'net').to_csv(
        result_dir + '/netflux_senMat_' + str(i) + '.csv'
    )
    res.estimate_sensitivity(which = 'total').to_csv(
        result_dir + '/totalflux_senMat_' + str(i) + '.csv'
    )






In [ ]:
import os

base_dir         = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/Fit_2/'
MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL_TEST.xlsx'
MEASURED_MDVS    = base_dir + '0_data/WT_10CU_MDV_TEST.xlsx'
# MEASURED_FLUXES  = base_dir + '0_data/WT_10CU_FLUX.xlsx'
MEASURED_FLUXES  = base_dir + '0_data/WT_10CU_FLUX_TEST.xlsx'
INI_FLUX         = base_dir + '0_data/WT_FBA_ref_TEST.xlsx'



results_fit_ini_2 = {}
obj_fit_ini_2     = {}



for i in range(31, 41):
    
    try:
       
        model = ff.Model('wt_10cu')
        model.read_from_file(MODEL_FILE)
        
        with model.fitter('ss') as fit:
            
            # specify the lableing strategy, use this method for every labeled substrate
            fit.set_labeling_strategy(
                'cpd01024.ex', 
                labeling_pattern = ['0', '1'], 
                percentage = [0.75, 0.25], 
                purity = [1, 1]
            )
    
            # set bounds for fluxes
            fit.set_flux_bounds('all', bounds = [-50, 50])
            fit.set_flux_bounds('o2in', bounds = [10, 30])
            fit.set_flux_bounds('ch4in', bounds = [10, 30])
            fit.set_flux_bounds('atpm', bounds = [15, 30])
            fit.set_flux_bounds('biom', bounds = [0.04, 0.2])
            
            # read measurements
            fit.set_measured_MDVs_from_file(MEASURED_MDVS)
            fit.set_measured_fluxes_from_file(MEASURED_FLUXES)
    
    
            
            # # slove the fluxes using initial flux guess by FBA
            # fit.prepare(n_jobs = 1)
            # res = fit.solve(
            #     solver = 'slsqp',
            #     ini_fluxes = INI_FLUX,
            #     fit_measured_fluxes = True,
            # )
    
            # slove the fluxes without reference
            fit.prepare(n_jobs = 1)
            res = fit.solve(
                solver = 'slsqp'
            )
    
            
        if res.opt_objective < 1000:

            # make directory
            result_dir = base_dir + str(i)
            os.mkdir(result_dir)
            
            results_fit_ini_2[str(i)] = res
            obj_fit_ini_2[str(i)] = res.opt_objective
        
            net_flux = pd.concat([  pd.Series(res.opt_net_fluxes)  ,  pd.Series([res.opt_objective], index = ['opt_objective'])  ])
            net_flux.to_csv(
                base_dir + '/estimated_net_fluxes_' + str(i) + '.csv'
            )
            # pd.Series(res.opt_net_fluxes).to_csv(
            #     base_dir + '/estimated_net_fluxes_' + str(i) + '.csv'
            # )
            pd.Series(res.opt_total_fluxes).to_csv(
                result_dir + '/estimated_total_fluxes_' + str(i) + '.csv'
            )
            
            # normal probability plot of residuals
            res.plot_normal_probability(show_fig = False, output_dir = result_dir)
            
            # compare simulations and measurements
            res.plot_simulated_vs_measured_MDVs(show_fig = False, output_dir = result_dir)
            res.plot_simulated_vs_measured_fluxes(show_fig = False, output_dir = result_dir)
            
            # export the contribution matrix
            res.estimate_contribution_matrix(which = 'net').to_csv(
                result_dir + '/netflux_contribMat_' + str(i) + '.csv'
            )
            res.estimate_contribution_matrix(which = 'total').to_csv(
                result_dir + '/totalflux_contribMat_' + str(i) + '.csv'
            )
            
            # export the sensitivity matrix
            res.estimate_sensitivity(which = 'net').to_csv(
                result_dir + '/netflux_senMat_' + str(i) + '.csv'
            )
            res.estimate_sensitivity(which = 'total').to_csv(
                result_dir + '/totalflux_senMat_' + str(i) + '.csv'
            )

    except:
        print('SVD converge fail: fit number ' + str(i))




In [ ]:
obj_fit_ini_2

In [ ]:
obj_fit_ini_2

In [ ]:
base_dir         = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/Fit_2/'
MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL_TEST.xlsx'
MEASURED_MDVS    = base_dir + '0_data/WT_10CU_MDV_TEST.xlsx'
# MEASURED_FLUXES  = base_dir + '0_data/WT_10CU_FLUX.xlsx'
MEASURED_FLUXES  = base_dir + '0_data/WT_10CU_FLUX_TEST.xlsx'
INI_FLUX         = base_dir + '0_data/WT_FBA_ref_TEST.xlsx'



results_fit_ini_2 = {}
obj_fit_ini_2     = {}



for i in range(71, 81):
    
    try:
       
        model = ff.Model('wt_10cu')
        model.read_from_file(MODEL_FILE)
        
        with model.fitter('ss') as fit:
            
            # specify the lableing strategy, use this method for every labeled substrate
            fit.set_labeling_strategy(
                'cpd01024.ex', 
                labeling_pattern = ['0', '1'], 
                percentage = [0.75, 0.25], 
                purity = [1, 1]
            )
    
            # set bounds for fluxes
            fit.set_flux_bounds('all', bounds = [-50, 50])
            fit.set_flux_bounds('o2in', bounds = [10, 30])
            fit.set_flux_bounds('ch4in', bounds = [10, 30])
            fit.set_flux_bounds('atpm', bounds = [2, 6])
            fit.set_flux_bounds('biom', bounds = [0.04, 0.2])
            
            # read measurements
            fit.set_measured_MDVs_from_file(MEASURED_MDVS)
            fit.set_measured_fluxes_from_file(MEASURED_FLUXES)
    
    
            
            # # slove the fluxes using initial flux guess by FBA
            # fit.prepare(n_jobs = 1)
            # res = fit.solve(
            #     solver = 'slsqp',
            #     ini_fluxes = INI_FLUX,
            #     fit_measured_fluxes = True,
            # )
    
            # slove the fluxes without reference
            fit.prepare(n_jobs = 1)
            res = fit.solve(
                solver = 'slsqp'
            )
    
            
        if res.opt_objective < 1000:

            # make directory
            result_dir = base_dir + str(i)
            os.mkdir(result_dir)
            
            results_fit_ini_2[str(i)] = res
            obj_fit_ini_2[str(i)] = res.opt_objective
        
            net_flux = pd.concat([  pd.Series(res.opt_net_fluxes)  ,  pd.Series([res.opt_objective], index = ['opt_objective'])  ])
            net_flux.to_csv(
                base_dir + '/estimated_net_fluxes_' + str(i) + '.csv'
            )
            # pd.Series(res.opt_net_fluxes).to_csv(
            #     base_dir + '/estimated_net_fluxes_' + str(i) + '.csv'
            # )
            pd.Series(res.opt_total_fluxes).to_csv(
                result_dir + '/estimated_total_fluxes_' + str(i) + '.csv'
            )
            
            # normal probability plot of residuals
            res.plot_normal_probability(show_fig = False, output_dir = result_dir)
            
            # compare simulations and measurements
            res.plot_simulated_vs_measured_MDVs(show_fig = False, output_dir = result_dir)
            res.plot_simulated_vs_measured_fluxes(show_fig = False, output_dir = result_dir)
            
            # export the contribution matrix
            res.estimate_contribution_matrix(which = 'net').to_csv(
                result_dir + '/netflux_contribMat_' + str(i) + '.csv'
            )
            res.estimate_contribution_matrix(which = 'total').to_csv(
                result_dir + '/totalflux_contribMat_' + str(i) + '.csv'
            )
            
            # export the sensitivity matrix
            res.estimate_sensitivity(which = 'net').to_csv(
                result_dir + '/netflux_senMat_' + str(i) + '.csv'
            )
            res.estimate_sensitivity(which = 'total').to_csv(
                result_dir + '/totalflux_senMat_' + str(i) + '.csv'
            )

    except:
        print('SVD converge fail: fit number ' + str(i))




In [ ]:
# Test FBA with ff_rxn_df
base_dir         = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/Fit_2/'
# MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL_TEST.xlsx'
MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL.xlsx'

ff_md = ff.Model('ff_model_test')
ff_md.read_from_file(MODEL_FILE)
with ff_md.optimizer() as opt:
    opt.set_flux_bounds('all', bounds = [-100, 100])
    # opt.set_flux_bounds('ch4in', bounds = [13.5, 15])
    opt.set_flux_bounds('o2in', bounds = [21.2, 21.2])
    opt.set_flux_bounds('atpm', bounds = [25.3, 25.3])
    # opt.set_flux_bounds('pyrin', bounds = [0, 2])
    # opt.set_flux_bounds('rxn00154_c0', bounds = [0, 1])

    # opt.set_flux_bounds('o2in', bounds = [23, 23])

    # opt.set_flux_bounds('rxn07847_c0', bounds = [0, 100])
    # opt.set_flux_bounds('rxn07848_c0', bounds = [0, 100])
    # opt.set_flux_bounds('rxn37236_c0', bounds = [0, 100])
    # opt.set_flux_bounds('rxn02480_c0', bounds = [0, 100])
    # opt.set_flux_bounds('rxn02431_c0', bounds = [0, 100])
    # opt.set_flux_bounds('rxn07849_c0', bounds = [0, 100])
    # opt.set_flux_bounds('rxn00371_c0', bounds = [0, 100])
    # opt.set_flux_bounds('rxn00248_c0', bounds = [-100, 0])

    # opt.set_flux_bounds('rxn00182_c0', bounds = [0, 0])
    # opt.set_flux_bounds('rxn00184_c0', bounds = [0, 0])
    # opt.set_flux_bounds('atps2', bounds = [0, 1])
    # opt.set_flux_bounds('biom', bounds = [0.10, 0.10])
    
    opt.prepare()
    res = opt.optimize(objective = {'biom': 1})
    # res = opt.optimize(objective = {'o2in': 1})

fluxes_pd_series = pd.Series(
    [res.opt_fluxes[rxn] for rxn in ff_md.reactions], name = 'fluxes', index = ff_md.reactions
)
fluxes_pd_series.to_excel(base_dir + '0_data/WT_FBA_ref_TEST.xlsx')
fluxes_pd_series.to_csv(base_dir + '0_data/WT_FBA_ref_TEST.csv')

# print('objective:  ',res.opt_objective)
print('growth:     ',res.opt_fluxes['biom'])
print('methane:    ',res.opt_fluxes['ch4in'])
print('oxygen:     ',res.opt_fluxes['o2in'])
# print('atpase_1:   ',res.opt_fluxes['atps1'])
# print('atpase_2:   ',res.opt_fluxes['atps2'])
# print('acetate_out:',res.opt_fluxes['acout'])

# print('pyruvate_in:',res.opt_fluxes['pyrin'])
# print('citrate_out:',res.opt_fluxes['citout'])
# print('succ_out:   ',res.opt_fluxes['succout'])
# print('fum_out:    ',res.opt_fluxes['fumout'])

# print('formate_out:',res.opt_fluxes['forout'])



In [ ]:
base_dir         = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/Fit_2/'
# MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL_TEST.xlsx'
# MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL.xlsx'
MODEL_FILE       = base_dir + '0_data/ff_rxn_df_test.xlsx'

# MEASURED_MDVS    = base_dir + '0_data/WT_10CU_MDV_TEST.xlsx'
MEASURED_MDVS    = base_dir + '0_data/test_MDVs.xlsx'

# MEASURED_FLUXES  = base_dir + '0_data/WT_10CU_FLUX_TEST.xlsx'
MEASURED_FLUXES  = base_dir + '0_data/test_fluxes.xlsx'


# INI_FLUX         = base_dir + '0_data/WT_FBA_ref_TEST.xlsx'
# INI_FLUX         = base_dir + '0_data/estimated_net_fluxes_65_BestFit.xlsx'

results_fit_ini_2 = {}
obj_fit_ini_2     = {}

for i in range(71, 72):
    
    try:
       
        model = ff.Model('wt_10cu')
        model.read_from_file(MODEL_FILE)
        
        with model.fitter('ss') as fit:
            
            # specify the lableing strategy, use this method for every labeled substrate
            fit.set_labeling_strategy(
                'cpd01024.ex', 
                labeling_pattern = ['0', '1'], 
                percentage = [0.75, 0.25], 
                purity = [1, 1]
            )
    
            # # set bounds for fluxes
            # fit.set_flux_bounds('all', bounds = [-50, 50])
            # fit.set_flux_bounds('o2in', bounds = [10, 30])
            # fit.set_flux_bounds('ch4in', bounds = [10, 30])
            # fit.set_flux_bounds('atpm', bounds = [15, 30])
            # fit.set_flux_bounds('biom', bounds = [0.04, 0.2])

            fit.set_flux_bounds('all', bounds = [-100, 100])
            fit.set_flux_bounds('ch4in', bounds = [13, 15])
            fit.set_flux_bounds('biom', bounds = [0.07, 0.11])
            
            # read measurements
            fit.set_measured_MDVs_from_file(MEASURED_MDVS)
            fit.set_measured_fluxes_from_file(MEASURED_FLUXES)
    
    
            
            # # slove the fluxes using initial flux guess by FBA
            # fit.prepare(n_jobs = 1)
            # res = fit.solve(
            #     solver = 'slsqp',
            #     ini_fluxes = INI_FLUX,
            #     fit_measured_fluxes = True,
            # )
    
            # slove the fluxes without reference
            fit.prepare(n_jobs = 1)
            res = fit.solve(
                solver = 'slsqp'
            )
    
        # print(str(i) + ': ' + str(res.opt_objective))
        
        # if res.opt_objective < 5000:

        # make directory
        result_dir = base_dir + str(i)
        os.mkdir(result_dir)
        
        results_fit_ini_2[str(i)] = res
        obj_fit_ini_2[str(i)] = res.opt_objective
    
        net_flux = pd.concat([  pd.Series(res.opt_net_fluxes)  ,  pd.Series([res.opt_objective], index = ['opt_objective'])  ])
        net_flux.to_csv(
            base_dir + '/estimated_net_fluxes_' + str(i) + '.csv'
        )
        # pd.Series(res.opt_net_fluxes).to_csv(
        #     base_dir + '/estimated_net_fluxes_' + str(i) + '.csv'
        # )
        pd.Series(res.opt_total_fluxes).to_csv(
            result_dir + '/estimated_total_fluxes_' + str(i) + '.csv'
        )
        
        # normal probability plot of residuals
        res.plot_normal_probability(show_fig = False, output_dir = result_dir)
        
        # compare simulations and measurements
        res.plot_simulated_vs_measured_MDVs(show_fig = False, output_dir = result_dir)
        res.plot_simulated_vs_measured_fluxes(show_fig = False, output_dir = result_dir)
        
        # export the contribution matrix
        res.estimate_contribution_matrix(which = 'net').to_csv(
            result_dir + '/netflux_contribMat_' + str(i) + '.csv'
        )
        res.estimate_contribution_matrix(which = 'total').to_csv(
            result_dir + '/totalflux_contribMat_' + str(i) + '.csv'
        )
        
        # export the sensitivity matrix
        res.estimate_sensitivity(which = 'net').to_csv(
            result_dir + '/netflux_senMat_' + str(i) + '.csv'
        )
        res.estimate_sensitivity(which = 'total').to_csv(
            result_dir + '/totalflux_senMat_' + str(i) + '.csv'
            )

    except:
        print('SVD converge fail: fit number ' + str(i))


In [ ]:
base_dir         = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/Fit_2/'
MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL_COPY.xlsx'
MEASURED_MDVS    = base_dir + '0_data/WT_10CU_MDV_TEST.xlsx'
MEASURED_FLUXES  = base_dir + '0_data/WT_10CU_FLUX_TEST.xlsx'
OUT_DIR          = base_dir + 'result_7'

# INI_FLUX         = base_dir + '0_data/WT_FBA_ref_TEST.xlsx'

def steady_state_fitting():

    wt = ff.Model('OB3b_10copper')
    wt.read_from_file(MODEL_FILE)
    
    with wt.fitter('ss') as fit:
        # specify the lableing strategy, 
        # use this method for every labeled substrate
        fit.set_labeling_strategy(
                'cpd01024.ex', 
                labeling_pattern = ['0', '1'], 
                percentage = [0.75, 0.25], 
                purity = [0.9999, 0.99]
            )
        
        # read measurements
        fit.set_measured_MDVs_from_file(MEASURED_MDVS)
        fit.set_measured_fluxes_from_file(MEASURED_FLUXES)
        
        # set upper and lower bounds for fluxes
        fit.set_flux_bounds('all', bounds = [-50, 50])
        # fit.set_flux_bounds('o2in', bounds = [10, 30])
        fit.set_flux_bounds('ch4in', bounds = [10, 30])
        fit.set_flux_bounds('atpm', bounds = [15, 30])
        fit.set_flux_bounds('biom', bounds = [0.07, 0.14])

        # solve the fluxes
        fit.prepare(n_jobs = 3)
        while True:
            res = fit.solve(solver = 'slsqp')
            if res.optimization_successful:
                break
    
    # save the results
    pd.Series(res.opt_net_fluxes).to_excel(
        OUT_DIR+'/estimated_net_fluxes.xlsx'
    )
    pd.Series(res.opt_total_fluxes).to_excel(
        OUT_DIR+'/estimated_total_fluxes.xlsx'
    )

    net_cis = res.estimate_confidence_intervals(
        which = 'net', 
        confidence_level = 0.95
    )
    pd.DataFrame(net_cis, index = ['LB', 'UB']).T.to_excel(
        OUT_DIR+'/netflux_le_CIs.xlsx'
    )
    
    total_cis = res.estimate_confidence_intervals(
        which = 'total', 
        confidence_level = 0.95
    )
    pd.DataFrame(total_cis, index = ['LB', 'UB']).T.to_excel(
        OUT_DIR+'/totalflux_le_CIs.xlsx'
    )

    # normal probability plot of residuals
    res.plot_normal_probability(show_fig = False, output_dir = OUT_DIR)
    
    # compare simulations and measurements
    res.plot_simulated_vs_measured_MDVs(show_fig = False, output_dir = OUT_DIR)
    res.plot_simulated_vs_measured_fluxes(show_fig = False, output_dir = OUT_DIR)
    
    # export the contribution matrix
    res.estimate_contribution_matrix(which = 'net').to_excel(
        OUT_DIR+'/netflux_contribMat.xlsx'
    )
    res.estimate_contribution_matrix(which = 'total').to_excel(
        OUT_DIR+'/totalflux_contribMat.xlsx'
    )
    
    # export the sensitivity matrix
    res.estimate_sensitivity(which = 'net').to_excel(
        OUT_DIR+'/netflux_senMat.xlsx'
    )
    res.estimate_sensitivity(which = 'total').to_excel(
        OUT_DIR+'/totalflux_senMat.xlsx'
    )

In [ ]:
steady_state_fitting()

In [ ]:
base_dir         = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/Fit_2/'
# MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL_TEST.xlsx'
# MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL.xlsx'
MODEL_FILE       = base_dir + 'ff_rxn_df_test.xlsx'

MEASURED_MDVS    = base_dir + '0_data/WT_10CU_MDV_TEST.xlsx'
MEASURED_FLUXES  = base_dir + '0_data/WT_10CU_FLUX_TEST.xlsx'
INI_FLUX         = base_dir + '0_data/WT_FBA_ref_TEST.xlsx'
# INI_FLUX         = base_dir + '0_data/estimated_net_fluxes_65_BestFit.xlsx'

results_fit_ini_2 = {}
obj_fit_ini_2     = {}

for i in range(71, 81):
    
    try:
       
        model = ff.Model('wt_10cu')
        model.read_from_file(MODEL_FILE)
        
        with model.fitter('ss') as fit:
            
            # specify the lableing strategy, use this method for every labeled substrate
            fit.set_labeling_strategy(
                'cpd01024.ex', 
                labeling_pattern = ['0', '1'], 
                percentage = [0.75, 0.25], 
                purity = [1, 1]
            )
    
            # set bounds for fluxes
            fit.set_flux_bounds('all', bounds = [-50, 50])
            fit.set_flux_bounds('o2in', bounds = [10, 30])
            fit.set_flux_bounds('ch4in', bounds = [10, 30])
            fit.set_flux_bounds('atpm', bounds = [15, 30])
            fit.set_flux_bounds('biom', bounds = [0.04, 0.2])
            
            # read measurements
            fit.set_measured_MDVs_from_file(MEASURED_MDVS)
            fit.set_measured_fluxes_from_file(MEASURED_FLUXES)
    
    
            
            # # slove the fluxes using initial flux guess by FBA
            # fit.prepare(n_jobs = 1)
            # res = fit.solve(
            #     solver = 'slsqp',
            #     ini_fluxes = INI_FLUX,
            #     fit_measured_fluxes = True,
            # )
    
            # slove the fluxes without reference
            fit.prepare(n_jobs = 1)
            res = fit.solve(
                solver = 'slsqp'
            )
    
        # print(str(i) + ': ' + str(res.opt_objective))
        
        # if res.opt_objective < 5000:

        # make directory
        result_dir = base_dir + str(i)
        os.mkdir(result_dir)
        
        results_fit_ini_2[str(i)] = res
        obj_fit_ini_2[str(i)] = res.opt_objective
    
        net_flux = pd.concat([  pd.Series(res.opt_net_fluxes)  ,  pd.Series([res.opt_objective], index = ['opt_objective'])  ])
        net_flux.to_csv(
            base_dir + '/estimated_net_fluxes_' + str(i) + '.csv'
        )
        # pd.Series(res.opt_net_fluxes).to_csv(
        #     base_dir + '/estimated_net_fluxes_' + str(i) + '.csv'
        # )
        pd.Series(res.opt_total_fluxes).to_csv(
            result_dir + '/estimated_total_fluxes_' + str(i) + '.csv'
        )
        
        # normal probability plot of residuals
        res.plot_normal_probability(show_fig = False, output_dir = result_dir)
        
        # compare simulations and measurements
        res.plot_simulated_vs_measured_MDVs(show_fig = False, output_dir = result_dir)
        res.plot_simulated_vs_measured_fluxes(show_fig = False, output_dir = result_dir)
        
        # export the contribution matrix
        res.estimate_contribution_matrix(which = 'net').to_csv(
            result_dir + '/netflux_contribMat_' + str(i) + '.csv'
        )
        res.estimate_contribution_matrix(which = 'total').to_csv(
            result_dir + '/totalflux_contribMat_' + str(i) + '.csv'
        )
        
        # export the sensitivity matrix
        res.estimate_sensitivity(which = 'net').to_csv(
            result_dir + '/netflux_senMat_' + str(i) + '.csv'
        )
        res.estimate_sensitivity(which = 'total').to_csv(
            result_dir + '/totalflux_senMat_' + str(i) + '.csv'
            )

    except:
        print('SVD converge fail: fit number ' + str(i))


In [ ]:

base_dir         = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/Fit_2/'
MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL_TEST.xlsx'
MEASURED_MDVS    = base_dir + '0_data/WT_10CU_MDV_TEST.xlsx'
# MEASURED_FLUXES  = base_dir + '0_data/WT_10CU_FLUX.xlsx'
MEASURED_FLUXES  = base_dir + '0_data/WT_10CU_FLUX_TEST.xlsx'
INI_FLUX         = base_dir + '0_data/WT_FBA_ref_TEST.xlsx'



results_fit_ini_2 = {}
obj_fit_ini_2     = {}



for i in range(71, 131):
    
    try:
       
        model = ff.Model('wt_10cu')
        model.read_from_file(MODEL_FILE)
        
        with model.fitter('ss') as fit:
            
            # specify the lableing strategy, use this method for every labeled substrate
            fit.set_labeling_strategy(
                'cpd01024.ex', 
                labeling_pattern = ['0', '1'], 
                percentage = [0.75, 0.25], 
                purity = [1, 1]
            )
    
            # set bounds for fluxes
            fit.set_flux_bounds('all', bounds = [-50, 50])
            fit.set_flux_bounds('o2in', bounds = [10, 30])
            fit.set_flux_bounds('ch4in', bounds = [10, 30])
            fit.set_flux_bounds('atpm', bounds = [15, 30])
            fit.set_flux_bounds('biom', bounds = [0.04, 0.2])
            
            # read measurements
            fit.set_measured_MDVs_from_file(MEASURED_MDVS)
            fit.set_measured_fluxes_from_file(MEASURED_FLUXES)
    
    
            
            # # slove the fluxes using initial flux guess by FBA
            # fit.prepare(n_jobs = 1)
            # res = fit.solve(
            #     solver = 'slsqp',
            #     ini_fluxes = INI_FLUX,
            #     fit_measured_fluxes = True,
            # )
    
            # slove the fluxes without reference
            fit.prepare(n_jobs = 1)
            res = fit.solve(
                solver = 'slsqp'
            )
    
            
        if res.opt_objective < 5000:

            # make directory
            result_dir = base_dir + str(i)
            os.mkdir(result_dir)
            
            results_fit_ini_2[str(i)] = res
            obj_fit_ini_2[str(i)] = res.opt_objective
        
            net_flux = pd.concat([  pd.Series(res.opt_net_fluxes)  ,  pd.Series([res.opt_objective], index = ['opt_objective'])  ])
            net_flux.to_csv(
                base_dir + '/estimated_net_fluxes_' + str(i) + '.csv'
            )
            # pd.Series(res.opt_net_fluxes).to_csv(
            #     base_dir + '/estimated_net_fluxes_' + str(i) + '.csv'
            # )
            pd.Series(res.opt_total_fluxes).to_csv(
                result_dir + '/estimated_total_fluxes_' + str(i) + '.csv'
            )
            
            # normal probability plot of residuals
            res.plot_normal_probability(show_fig = False, output_dir = result_dir)
            
            # compare simulations and measurements
            res.plot_simulated_vs_measured_MDVs(show_fig = False, output_dir = result_dir)
            res.plot_simulated_vs_measured_fluxes(show_fig = False, output_dir = result_dir)
            
            # export the contribution matrix
            res.estimate_contribution_matrix(which = 'net').to_csv(
                result_dir + '/netflux_contribMat_' + str(i) + '.csv'
            )
            res.estimate_contribution_matrix(which = 'total').to_csv(
                result_dir + '/totalflux_contribMat_' + str(i) + '.csv'
            )
            
            # export the sensitivity matrix
            res.estimate_sensitivity(which = 'net').to_csv(
                result_dir + '/netflux_senMat_' + str(i) + '.csv'
            )
            res.estimate_sensitivity(which = 'total').to_csv(
                result_dir + '/totalflux_senMat_' + str(i) + '.csv'
            )

    except:
        print('SVD converge fail: fit number ' + str(i))




In [ ]:
base_dir         = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/Fit_3/'
MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL_TEST.xlsx'
MEASURED_MDVS    = base_dir + '0_data/WT_10CU_MDV_TEST.xlsx'
MEASURED_FLUXES  = base_dir + '0_data/WT_10CU_FLUX_TEST.xlsx'
INI_FLUX         = base_dir + '0_data/WT_FBA_ref_TEST.xlsx'



results_fit_ini_3 = {}
obj_fit_ini_3     = {}

for i in range(1, 11):
    
    try:
       
        model = ff.Model('wt_10cu')
        model.read_from_file(MODEL_FILE)
        
        with model.fitter('ss') as fit:
            
            # specify the lableing strategy, use this method for every labeled substrate
            fit.set_labeling_strategy(
                'cpd01024.ex', 
                labeling_pattern = ['0', '1'], 
                percentage = [0.75, 0.25], 
                purity = [1, 1]
            )
    
            # set bounds for fluxes
            fit.set_flux_bounds('all', bounds = [-50, 50])
            fit.set_flux_bounds('o2in', bounds = [10, 30])
            fit.set_flux_bounds('ch4in', bounds = [0, 30])
            fit.set_flux_bounds('atpm', bounds = [15, 30])
            fit.set_flux_bounds('biom', bounds = [0.04, 0.2])
            
            # read measurements
            fit.set_measured_MDVs_from_file(MEASURED_MDVS)
            fit.set_measured_fluxes_from_file(MEASURED_FLUXES)
    
           
            # # slove the fluxes using initial flux guess by FBA
            # fit.prepare(n_jobs = 1)
            # res = fit.solve(
            #     solver = 'slsqp',
            #     ini_fluxes = INI_FLUX,
            #     fit_measured_fluxes = True,
            # )
    
            # slove the fluxes without reference
            fit.prepare(n_jobs = 1)
            res = fit.solve(
                solver = 'slsqp'
            )
    
        print(str(i) + ': ' + str(res.opt_objective))

        # make directory
        result_dir = base_dir + str(i)
        os.mkdir(result_dir)
        
        results_fit_ini_3[str(i)] = res
        obj_fit_ini_3[str(i)] = res.opt_objective
    
        net_flux = pd.concat([  pd.Series(res.opt_net_fluxes)  ,  pd.Series([res.opt_objective], index = ['opt_objective'])  ])
        net_flux.to_csv(
            base_dir + '/estimated_net_fluxes_' + str(i) + '.csv'
        )
        # pd.Series(res.opt_net_fluxes).to_csv(
        #     base_dir + '/estimated_net_fluxes_' + str(i) + '.csv'
        # )
        pd.Series(res.opt_total_fluxes).to_csv(
            result_dir + '/estimated_total_fluxes_' + str(i) + '.csv'
        )
        
        # normal probability plot of residuals
        res.plot_normal_probability(show_fig = False, output_dir = result_dir)
        
        # compare simulations and measurements
        res.plot_simulated_vs_measured_MDVs(show_fig = False, output_dir = result_dir)
        res.plot_simulated_vs_measured_fluxes(show_fig = False, output_dir = result_dir)
        
        # export the contribution matrix
        res.estimate_contribution_matrix(which = 'net').to_csv(
            result_dir + '/netflux_contribMat_' + str(i) + '.csv'
        )
        res.estimate_contribution_matrix(which = 'total').to_csv(
            result_dir + '/totalflux_contribMat_' + str(i) + '.csv'
        )
        
        # export the sensitivity matrix
        res.estimate_sensitivity(which = 'net').to_csv(
            result_dir + '/netflux_senMat_' + str(i) + '.csv'
        )
        res.estimate_sensitivity(which = 'total').to_csv(
            result_dir + '/totalflux_senMat_' + str(i) + '.csv'
        )


    
        # if res.opt_objective < 5000:

        #     # make directory
        #     result_dir = base_dir + str(i)
        #     os.mkdir(result_dir)
            
        #     results_fit_ini_2[str(i)] = res
        #     obj_fit_ini_2[str(i)] = res.opt_objective
        
        #     net_flux = pd.concat([  pd.Series(res.opt_net_fluxes)  ,  pd.Series([res.opt_objective], index = ['opt_objective'])  ])
        #     net_flux.to_csv(
        #         base_dir + '/estimated_net_fluxes_' + str(i) + '.csv'
        #     )
        #     # pd.Series(res.opt_net_fluxes).to_csv(
        #     #     base_dir + '/estimated_net_fluxes_' + str(i) + '.csv'
        #     # )
        #     pd.Series(res.opt_total_fluxes).to_csv(
        #         result_dir + '/estimated_total_fluxes_' + str(i) + '.csv'
        #     )
            
        #     # normal probability plot of residuals
        #     res.plot_normal_probability(show_fig = False, output_dir = result_dir)
            
        #     # compare simulations and measurements
        #     res.plot_simulated_vs_measured_MDVs(show_fig = False, output_dir = result_dir)
        #     res.plot_simulated_vs_measured_fluxes(show_fig = False, output_dir = result_dir)
            
        #     # export the contribution matrix
        #     res.estimate_contribution_matrix(which = 'net').to_csv(
        #         result_dir + '/netflux_contribMat_' + str(i) + '.csv'
        #     )
        #     res.estimate_contribution_matrix(which = 'total').to_csv(
        #         result_dir + '/totalflux_contribMat_' + str(i) + '.csv'
        #     )
            
        #     # export the sensitivity matrix
        #     res.estimate_sensitivity(which = 'net').to_csv(
        #         result_dir + '/netflux_senMat_' + str(i) + '.csv'
        #     )
        #     res.estimate_sensitivity(which = 'total').to_csv(
        #         result_dir + '/totalflux_senMat_' + str(i) + '.csv'
        #     )

    except:
        print('SVD converge fail: fit number ' + str(i))


In [ ]:
for i in range(41, 44):
    
    try:
       
        model = ff.Model('wt_10cu')
        model.read_from_file(MODEL_FILE)
        
        with model.fitter('ss') as fit:
            
            # specify the lableing strategy, use this method for every labeled substrate
            fit.set_labeling_strategy(
                'cpd01024.ex', 
                labeling_pattern = ['0', '1'], 
                percentage = [0.75, 0.25], 
                purity = [1, 1]
            )
    
            # set bounds for fluxes
            fit.set_flux_bounds('all', bounds = [-50, 50])
            fit.set_flux_bounds('o2in', bounds = [10, 30])
            fit.set_flux_bounds('ch4in', bounds = [10, 30])
            fit.set_flux_bounds('atpm', bounds = [2, 6])
            fit.set_flux_bounds('biom', bounds = [0.04, 0.2])
            
            # read measurements
            fit.set_measured_MDVs_from_file(MEASURED_MDVS)
            fit.set_measured_fluxes_from_file(MEASURED_FLUXES)
    
    
            
            # slove the fluxes using initial flux guess by FBA
            fit.prepare(n_jobs = 1)
            res = fit.solve(
                solver = 'slsqp',
                ini_fluxes = INI_FLUX,
                fit_measured_fluxes = True,
            )
    
            # # slove the fluxes without reference
            # fit.prepare(n_jobs = 1)
            # res = fit.solve(
            #     solver = 'slsqp'
            # )
    
        print(res.opt_objective)
        
        if res.opt_objective < 1000:

            # make directory
            result_dir = base_dir + str(i)
            os.mkdir(result_dir)
            
            results_fit_ini_1[str(i)] = res
            obj_fit_ini_1[str(i)] = res.opt_objective
        
            net_flux = pd.concat([  pd.Series(res.opt_net_fluxes)  ,  pd.Series([res.opt_objective], index = ['opt_objective'])  ])
            net_flux.to_csv(
                base_dir + '/estimated_net_fluxes_' + str(i) + '.csv'
            )
            # pd.Series(res.opt_net_fluxes).to_csv(
            #     base_dir + '/estimated_net_fluxes_' + str(i) + '.csv'
            # )
            pd.Series(res.opt_total_fluxes).to_csv(
                result_dir + '/estimated_total_fluxes_' + str(i) + '.csv'
            )
            
            # normal probability plot of residuals
            res.plot_normal_probability(show_fig = False, output_dir = result_dir)
            
            # compare simulations and measurements
            res.plot_simulated_vs_measured_MDVs(show_fig = False, output_dir = result_dir)
            res.plot_simulated_vs_measured_fluxes(show_fig = False, output_dir = result_dir)
            
            # export the contribution matrix
            res.estimate_contribution_matrix(which = 'net').to_csv(
                result_dir + '/netflux_contribMat_' + str(i) + '.csv'
            )
            res.estimate_contribution_matrix(which = 'total').to_csv(
                result_dir + '/totalflux_contribMat_' + str(i) + '.csv'
            )
            
            # export the sensitivity matrix
            res.estimate_sensitivity(which = 'net').to_csv(
                result_dir + '/netflux_senMat_' + str(i) + '.csv'
            )
            res.estimate_sensitivity(which = 'total').to_csv(
                result_dir + '/totalflux_senMat_' + str(i) + '.csv'
            )

    except:
        print('SVD converge fail: fit number ' + str(i))


In [ ]:
res.opt_net_fluxes['opt_objective'] = obj_fit_ini_2[0]

In [ ]:
test = pd.Series(res.opt_net_fluxes)

In [ ]:
test.append(pd.Series([res.opt_objective], index = ['opt_objective']))

In [ ]:
WT_Dir = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/'

MODEL_FILE = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/Good_Fit_1/Data/WT_10CU_MODEL_TEST.xlsx'

# MEASURED_MDVS = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/WT_10CU_MDV.xlsx'
MEASURED_MDVS = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/Good_Fit_1/Data/WT_10CU_MDV_TEST.xlsx'

MEASURED_FLUXES = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/Good_Fit_1/Data/WT_10CU_FLUX.xlsx'

# INI_FLUX = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/WT_FBA_ref.xlsx'
INI_FLUX = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/Good_Fit_1/Data/WT_FBA_ref_TEST.xlsx'

# OUT_DIR = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/Good_Fit_1/'
OUT_DIR = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/WT_results/'


# estimate fluxes at steady state
def wt_model_steady_state_fitting():
    
    model = ff.Model('wt_10cu')
    model.read_from_file(MODEL_FILE)
    
    with model.fitter('ss') as fit:
        # specify the lableing strategy, 
        # use this method for every labeled substrate
        fit.set_labeling_strategy(
            'cpd01024.ex', 
            labeling_pattern = ['0', '1'], 
            percentage = [0.75, 0.25], 
            purity = [1, 1]
        )
        
        # set bounds for fluxes
        fit.set_flux_bounds('all', bounds = [-50, 50])
        fit.set_flux_bounds('o2in', bounds = [10, 30])
        fit.set_flux_bounds('ch4in', bounds = [10, 30])
        fit.set_flux_bounds('atpm', bounds = [20, 30])
        fit.set_flux_bounds('biom', bounds = [0.04, 0.2])
        
        # read measurements
        fit.set_measured_MDVs_from_file(MEASURED_MDVS)
        fit.set_measured_fluxes_from_file(MEASURED_FLUXES)

        # slove the fluxes
        # fit.prepare(n_jobs = 1)
        # res = fit.solve(
        #     solver = 'slsqp',
        #     ini_fluxes = INI_FLUX,
        #     fit_measured_fluxes = True,
        # )
        
        # slove the fluxes
        fit.prepare(n_jobs = 1)
        res = fit.solve(
            solver = 'slsqp'
        )
    # print(res.optimization_successful)
    
    # save the results
    # pd.Series(res.opt_net_fluxes).to_excel(
    #     OUT_DIR+'/estimated_net_fluxes.xlsx'
    # )
    pd.Series(res.opt_net_fluxes).to_csv(
        OUT_DIR+'/estimated_net_fluxes.csv'
    )
    pd.Series(res.opt_total_fluxes).to_excel(
        OUT_DIR+'/estimated_total_fluxes.xlsx'
    )
    
    # normal probability plot of residuals
    res.plot_normal_probability(show_fig = False, output_dir = OUT_DIR)
    
    # compare simulations and measurements
    res.plot_simulated_vs_measured_MDVs(show_fig = False, output_dir = OUT_DIR)
    res.plot_simulated_vs_measured_fluxes(show_fig = False, output_dir = OUT_DIR)
    
    # export the contribution matrix
    res.estimate_contribution_matrix(which = 'net').to_excel(
        OUT_DIR+'/netflux_contribMat.xlsx'
    )
    res.estimate_contribution_matrix(which = 'total').to_excel(
        OUT_DIR+'/totalflux_contribMat.xlsx'
    )
    
    # export the sensitivity matrix
    res.estimate_sensitivity(which = 'net').to_excel(
        OUT_DIR+'/netflux_senMat.xlsx'
    )
    res.estimate_sensitivity(which = 'total').to_excel(
        OUT_DIR+'/totalflux_senMat.xlsx'
    )





In [ ]:
wt_model_steady_state_fitting()

In [ ]:
# estimate with confidence intervals
def wt_model_steady_state_fitting_CIs():

    model = ff.Model('demo')
    model.read_from_file(MODEL_FILE)
    
    with model.fitter('ss') as fit:
        # specify the lableing strategy, 
        # use this method for every labeled substrate
        fit.set_labeling_strategy(
            'cpd01024.ex', 
            labeling_pattern = ['0', '1'], 
            percentage = [0.75, 0.25], 
            purity = [1, 1]
        )
        
        # set upper and lower bounds for fluxes
        fit.set_flux_bounds('all', bounds = [-100, 100])
        fit.set_flux_bounds('ch4in', bounds = [13, 15])
        fit.set_flux_bounds('rxn00182_c0', bounds = [0, 0.0001])
        fit.set_flux_bounds('rxn00184_c0', bounds = [0, 0.0001])
    
        fit.set_flux_bounds('biom', bounds = [0.09, 0.11])
        
        # read measurements
        fit.set_measured_MDVs_from_file(MEASURED_MDVS)
        fit.set_measured_fluxes_from_file(MEASURED_FLUXES)
        
        # estimate the confidence intervals
        fit.prepare(n_jobs = 3)
        res = fit.solve_with_confidence_intervals(
            solver = 'slsqp', 
            n_runs = 100, 
            n_jobs = 3
        )
    
    # save the CIs
    net_cis = res.estimate_confidence_intervals(
        which = 'net', 
        confidence_level = 0.95
    )
    pd.DataFrame(net_cis, index = ['LB', 'UB']).T.to_excel(
        OUT_DIR+'/netflux_CIs.xlsx'
    )

    total_cis = res.estimate_confidence_intervals(
        which = 'total', 
        confidence_level = 0.95
    )
    pd.DataFrame(total_cis, index = ['LB', 'UB']).T.to_excel(
        OUT_DIR+'/totalflux_CIs.xlsx'
    )




# if __name__ == '__main__':

#     makedirs(OUT_DIR, exist_ok = True)
#     toy_model_steady_state_fitting()
#     toy_model_steady_state_fitting_CIs()

In [ ]:
fluxes_pd_series = pd.Series(
    [res.opt_fluxes[rxn] for rxn in ff_md.reactions], name = 'fluxes', index = ff_md.reactions
)

fluxes_pd_series.to_csv('ff_fba.csv')

In [ ]:
print('growth:     ',res.opt_fluxes['biom'])
print('methane:    ',res.opt_fluxes['ch4in'])
print('oxygen:     ',res.opt_fluxes['o2in'])
print('atpase_1:   ',res.opt_fluxes['atps1'])
print('atpase_2:   ',res.opt_fluxes['atps2'])
print('acetate_out:',res.opt_fluxes['acout'])

In [ ]:
MODEL_FILE = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/ff_rxn_df_test.xlsx'
FLUXES_FILE = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/ff_fba_test.xlsx'

ff_md = ff.Model('ff_model_test')
ff_md.read_from_file(MODEL_FILE)
with ff_md.simulator('ss') as sim:
    sim.set_target_EMUs({
        'cpd00022': '12',
        'cpd00130': '1234',
        'cpd00035': '123',
        'cpd00036': '1234',
        'cpd00054': '123',
        'cpd00137': '12345'
    })
    sim.set_labeling_strategy(
        'cpd01024.ex',
        labeling_pattern = ['1'],
        percentage = [0.25],
        purity = [0.998]
    )
    sim.set_fluxes_from_file(FLUXES_FILE)
    sim.prepare(n_jobs = 1)
    res = sim.simulate()
print(res)

In [ ]:
MODEL_FILE = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/ff_rxn_df_test.xlsx'
FLUXES_FILE = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/ff_fba_test.xlsx'

ff_md = ff.Model('ff_model_test')
ff_md.read_from_file(MODEL_FILE)
with ff_md.simulator('ss') as sim:
    sim.set_target_EMUs({
        'cpd00022': '12',
        'cpd00130': '1234',
        'cpd00035': '123',
        'cpd00036': '1234',
        'cpd00054': '123',
        'cpd00137': '12345'
    })
    sim.set_labeling_strategy(
        'cpd01024.ex',
        labeling_pattern = ['1'],
        percentage = [1],
        purity = [0.998]
    )
    sim.set_fluxes_from_file(FLUXES_FILE)
    sim.prepare(n_jobs = 1)
    res = sim.simulate()
print(res)

In [ ]:
MODEL_FILE = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/ff_rxn_df_test.xlsx'
FLUXES_FILE = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/ff_fba_test_2.xlsx'

ff_md = ff.Model('ff_model_test')
ff_md.read_from_file(MODEL_FILE)
with ff_md.simulator('ss') as sim:
    sim.set_target_EMUs({
        'cpd00022': '12',
        'cpd00130': '1234',
        'cpd00035': '123',
        'cpd00036': '1234',
        'cpd00054': '123',
        'cpd00137': '12345'
    })
    sim.set_labeling_strategy(
        'cpd01024.ex',
        labeling_pattern = ['1'],
        percentage = [0.25],
        purity = [0.998]
    )
    sim.set_fluxes_from_file(FLUXES_FILE)
    sim.prepare(n_jobs = 1)
    res = sim.simulate()
print(res)

In [ ]:
# MODEL_FILE = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/ff_rxn_df_test.xlsx'
# MEASURED_MDVS = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/test_MDVs.xlsx'
# MEASURED_FLUXES = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/test_fluxes.xlsx'
# OUT_DIR = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/results_updated_njob3'


base_dir         = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/Fit_2/'
MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL_COPY.xlsx'
# MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL_TEST.xlsx'
# MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL.xlsx'
# MODEL_FILE       = base_dir + '0_data/ff_rxn_df_test.xlsx'

MEASURED_MDVS    = base_dir + '0_data/WT_10CU_MDV_TEST.xlsx'
# MEASURED_MDVS    = base_dir + '0_data/test_MDVs.xlsx'

MEASURED_FLUXES  = base_dir + '0_data/WT_10CU_FLUX_TEST.xlsx'
# MEASURED_FLUXES  = base_dir + '0_data/test_fluxes.xlsx'

# OUT_DIR = base_dir + 'results3'


# estimate fluxes at steady state
def toy_model_steady_state_fitting():
    
    model = ff.Model('demo')
    model.read_from_file(MODEL_FILE)
    
    with model.fitter('ss') as fit:
        # specify the lableing strategy, 
        # use this method for every labeled substrate
        fit.set_labeling_strategy(
            'cpd01024.ex', 
            labeling_pattern = ['0', '1'], 
            percentage = [0.75, 0.25], 
            purity = [1, 1]
        )
        
        # set bounds for fluxes
        fit.set_flux_bounds('all', bounds = [-100, 100])
        fit.set_flux_bounds('ch4in', bounds = [10, 25])
        # fit.set_flux_bounds('rxn00182_c0', bounds = [0, 0])
        # fit.set_flux_bounds('rxn00184_c0', bounds = [0, 0])
    
        fit.set_flux_bounds('biom', bounds = [0.07, 0.14])
        
        # read measurements
        fit.set_measured_MDVs_from_file(MEASURED_MDVS)
        fit.set_measured_fluxes_from_file(MEASURED_FLUXES)

        # slove the fluxes
        fit.prepare(n_jobs = 1)
        res = fit.solve(solver = 'slsqp')
    
    # print(res.optimization_successful)
    
    # save the results
    pd.Series(res.opt_net_fluxes).to_excel(
        OUT_DIR+'/estimated_net_fluxes.xlsx'
    )
    pd.Series(res.opt_total_fluxes).to_excel(
        OUT_DIR+'/estimated_total_fluxes.xlsx'
    )
    
    # normal probability plot of residuals
    res.plot_normal_probability(show_fig = False, output_dir = OUT_DIR)
    
    # compare simulations and measurements
    res.plot_simulated_vs_measured_MDVs(show_fig = False, output_dir = OUT_DIR)
    res.plot_simulated_vs_measured_fluxes(show_fig = False, output_dir = OUT_DIR)
    
    # export the contribution matrix
    res.estimate_contribution_matrix(which = 'net').to_excel(
        OUT_DIR+'/netflux_contribMat.xlsx'
    )
    res.estimate_contribution_matrix(which = 'total').to_excel(
        OUT_DIR+'/totalflux_contribMat.xlsx'
    )
    
    # export the sensitivity matrix
    res.estimate_sensitivity(which = 'net').to_excel(
        OUT_DIR+'/netflux_senMat.xlsx'
    )
    res.estimate_sensitivity(which = 'total').to_excel(
        OUT_DIR+'/totalflux_senMat.xlsx'
    )


In [ ]:
# MODEL_FILE = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/ff_rxn_df_test.xlsx'
# MEASURED_MDVS = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/test_MDVs.xlsx'
# MEASURED_FLUXES = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/test_fluxes.xlsx'
# OUT_DIR = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/results_updated_njob3'


base_dir         = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/Fit_2/'
MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL_COPY.xlsx'
# MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL_TEST.xlsx'
# MODEL_FILE       = base_dir + '0_data/WT_10CU_MODEL.xlsx'
# MODEL_FILE       = base_dir + '0_data/ff_rxn_df_test.xlsx'

MEASURED_MDVS    = base_dir + '0_data/WT_10CU_MDV_TEST.xlsx'
# MEASURED_MDVS    = base_dir + '0_data/test_MDVs.xlsx'

MEASURED_FLUXES  = base_dir + '0_data/WT_10CU_FLUX_TEST.xlsx'
# MEASURED_FLUXES  = base_dir + '0_data/test_fluxes.xlsx'

# OUT_DIR = base_dir + 'results3'


# estimate fluxes at steady state
def toy_model_steady_state_fitting():
    
    model = ff.Model('demo')
    model.read_from_file(MODEL_FILE)
    
    fit = model.fitter('ss')
    # specify the lableing strategy, 
    # use this method for every labeled substrate
    fit.set_labeling_strategy(
        'cpd01024.ex', 
        labeling_pattern = ['0', '1'], 
        percentage = [0.75, 0.25], 
        purity = [1, 1]
    )
    
    # set bounds for fluxes
    fit.set_flux_bounds('all', bounds = [-100, 100])
    fit.set_flux_bounds('ch4in', bounds = [10, 25])
    # fit.set_flux_bounds('rxn00182_c0', bounds = [0, 0])
    # fit.set_flux_bounds('rxn00184_c0', bounds = [0, 0])

    fit.set_flux_bounds('biom', bounds = [0.07, 0.14])
    
    # read measurements
    fit.set_measured_MDVs_from_file(MEASURED_MDVS)
    fit.set_measured_fluxes_from_file(MEASURED_FLUXES)

    # slove the fluxes
    fit.prepare(n_jobs = 1)
    res = fit.solve(solver = 'slsqp')

    # print(res.optimization_successful)
    
    # save the results
    pd.Series(res.opt_net_fluxes).to_excel(
        OUT_DIR+'/estimated_net_fluxes.xlsx'
    )
    pd.Series(res.opt_total_fluxes).to_excel(
        OUT_DIR+'/estimated_total_fluxes.xlsx'
    )
    
    # normal probability plot of residuals
    res.plot_normal_probability(show_fig = False, output_dir = OUT_DIR)
    
    # compare simulations and measurements
    res.plot_simulated_vs_measured_MDVs(show_fig = False, output_dir = OUT_DIR)
    res.plot_simulated_vs_measured_fluxes(show_fig = False, output_dir = OUT_DIR)
    
    # export the contribution matrix
    res.estimate_contribution_matrix(which = 'net').to_excel(
        OUT_DIR+'/netflux_contribMat.xlsx'
    )
    res.estimate_contribution_matrix(which = 'total').to_excel(
        OUT_DIR+'/totalflux_contribMat.xlsx'
    )
    
    # export the sensitivity matrix
    res.estimate_sensitivity(which = 'net').to_excel(
        OUT_DIR+'/netflux_senMat.xlsx'
    )
    res.estimate_sensitivity(which = 'total').to_excel(
        OUT_DIR+'/totalflux_senMat.xlsx'
    )


In [ ]:
OUT_DIR = base_dir + 'result_6'
toy_model_steady_state_fitting()

In [ ]:
res.opt_net_fluxes

In [ ]:
res.opt_objective

In [ ]:
toy_model_steady_state_fitting()

In [ ]:
MODEL_FILE = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/13CFLUX_SIMULATION/WT_SIMULATION/WT_10CU_MODEL.xlsx'
MEASURED_MDVS = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/test_MDVs.xlsx'
MEASURED_FLUXES = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/test_fluxes.xlsx'
OUT_DIR = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/results_updated_njob3'


# estimate fluxes at steady state
def toy_model_steady_state_fitting():
    
    model = ff.Model('demo')
    model.read_from_file(MODEL_FILE)
    
    with model.fitter('ss') as fit:
        # specify the lableing strategy, 
        # use this method for every labeled substrate
        fit.set_labeling_strategy(
            'cpd01024.ex', 
            labeling_pattern = ['0', '1'], 
            percentage = [0.75, 0.25], 
            purity = [1, 1]
        )
        
        # set bounds for fluxes
        fit.set_flux_bounds('all', bounds = [-100, 100])
        fit.set_flux_bounds('ch4in', bounds = [13, 15])
        # fit.set_flux_bounds('rxn00182_c0', bounds = [0, 0])
        # fit.set_flux_bounds('rxn00184_c0', bounds = [0, 0])
    
        fit.set_flux_bounds('biom', bounds = [0.07, 0.11])
        
        # read measurements
        fit.set_measured_MDVs_from_file(MEASURED_MDVS)
        fit.set_measured_fluxes_from_file(MEASURED_FLUXES)

        # slove the fluxes
        fit.prepare(n_jobs = 2)
        res = fit.solve(solver = 'slsqp')
    
    # print(res.optimization_successful)
    
    # save the results
    pd.Series(res.opt_net_fluxes).to_excel(
        OUT_DIR+'/estimated_net_fluxes.xlsx'
    )
    pd.Series(res.opt_total_fluxes).to_excel(
        OUT_DIR+'/estimated_total_fluxes.xlsx'
    )
    
    # normal probability plot of residuals
    res.plot_normal_probability(show_fig = False, output_dir = OUT_DIR)
    
    # compare simulations and measurements
    res.plot_simulated_vs_measured_MDVs(show_fig = False, output_dir = OUT_DIR)
    res.plot_simulated_vs_measured_fluxes(show_fig = False, output_dir = OUT_DIR)
    
    # export the contribution matrix
    res.estimate_contribution_matrix(which = 'net').to_excel(
        OUT_DIR+'/netflux_contribMat.xlsx'
    )
    res.estimate_contribution_matrix(which = 'total').to_excel(
        OUT_DIR+'/totalflux_contribMat.xlsx'
    )
    
    # export the sensitivity matrix
    res.estimate_sensitivity(which = 'net').to_excel(
        OUT_DIR+'/netflux_senMat.xlsx'
    )
    res.estimate_sensitivity(which = 'total').to_excel(
        OUT_DIR+'/totalflux_senMat.xlsx'
    )


# estimate with confidence intervals
def toy_model_steady_state_fitting_CIs():

    model = ff.Model('demo')
    model.read_from_file(MODEL_FILE)
    
    with model.fitter('ss') as fit:
        # specify the lableing strategy, 
        # use this method for every labeled substrate
        fit.set_labeling_strategy(
            'cpd01024.ex', 
            labeling_pattern = ['0', '1'], 
            percentage = [0.75, 0.25], 
            purity = [1, 1]
        )
        
        # set upper and lower bounds for fluxes
        fit.set_flux_bounds('all', bounds = [-100, 100])
        fit.set_flux_bounds('ch4in', bounds = [13, 15])
        fit.set_flux_bounds('rxn00182_c0', bounds = [0, 0.0001])
        fit.set_flux_bounds('rxn00184_c0', bounds = [0, 0.0001])
    
        fit.set_flux_bounds('biom', bounds = [0.09, 0.11])
        
        # read measurements
        fit.set_measured_MDVs_from_file(MEASURED_MDVS)
        fit.set_measured_fluxes_from_file(MEASURED_FLUXES)
        
        # estimate the confidence intervals
        fit.prepare(n_jobs = 2)
        res = fit.solve_with_confidence_intervals(
            solver = 'slsqp', 
            n_runs = 100, 
            n_jobs = 3
        )
    
    # save the CIs
    net_cis = res.estimate_confidence_intervals(
        which = 'net', 
        confidence_level = 0.95
    )
    pd.DataFrame(net_cis, index = ['LB', 'UB']).T.to_excel(
        OUT_DIR+'/netflux_CIs.xlsx'
    )

    total_cis = res.estimate_confidence_intervals(
        which = 'total', 
        confidence_level = 0.95
    )
    pd.DataFrame(total_cis, index = ['LB', 'UB']).T.to_excel(
        OUT_DIR+'/totalflux_CIs.xlsx'
    )




# if __name__ == '__main__':

#     makedirs(OUT_DIR, exist_ok = True)
#     toy_model_steady_state_fitting()
#     toy_model_steady_state_fitting_CIs()

In [ ]:
toy_model_steady_state_fitting()

In [ ]:
toy_model_steady_state_fitting()

In [ ]:
toy_model_steady_state_fitting_CIs()

In [ ]:
arg_syn = ['rxn00192_c0', 'rxn01917_c0', 'rxn02465_c0', 'rxn01637_c0', 
           'rxn00469_c0', 'rxn01019_c0', 'rxn01434_c0', 'rxn00802_c0']

flip_reaction([md.reactions.rxn01637_c0])

lump_path_dict = {'arg_synthesis': arg_syn}

for name, path in lump_path_dict.items():

    path_rxn_list = [md.reactions.get_by_id(rxn) for rxn in path]
    
    lumped_cpd_coeff_dict = {}
    
    for rxn in path_rxn_list:
        met_list = list(rxn.metabolites.keys())
        for met, coeff in rxn.metabolites.items():
            if met in list(lumped_cpd_coeff_dict.keys()):
                lumped_cpd_coeff_dict[met] += coeff
            else:
                lumped_cpd_coeff_dict[met] = coeff

    del_cpd = [cpd for cpd, coeff in lumped_cpd_coeff_dict.items() if coeff == 0]
    for cpd in del_cpd:
        lumped_cpd_coeff_dict.pop(cpd, None)
    
    lumped_rxn = cobra.Reaction(name)
    lumped_rxn.name = name
    lumped_rxn.bounds = (0, 1000)
    lumped_rxn.add_metabolites(lumped_cpd_coeff_dict)
    md.add_reactions([lumped_rxn])

md.reactions.arg_synthesis
        
        
        
            
    


In [ ]:
md.metabolites.cpd00023_c0.formula_weight

In [ ]:
md.reactions.rxn00786_c0

In [ ]:
md.reactions.rxn00182_c0.metabolites.keys()

In [ ]:
test = []
test.append(md.metabolites.cpd00001_c0)
test.append(md.metabolites.cpd00002_c0)
test.append(md.metabolites.cpd00003_c0)
test.append(md.metabolites.cpd00004_c0)
test

In [ ]:
tm = md.metabolites.cpd00008_c0
tm in test

In [ ]:
met = md.metabolites.cpd00087_c0
met_formula = met.formula
carbon_label = 'abcdefghijklmnopqrstuvwxyz'

met_unpack = [s for s in met_formula]
met_re = re.split('(\d+)',met_formula)

try:
    c = met_unpack.index('C') + 1
except:
    c = 0
print(c)
if c != 0: 
    try:
        c_num_ind = met_re.index('C')+1
        c_num = int(met_re[c_num_ind])
    except:
        c_num = 1
else:
    c_num = 0 
print(c_num)
if c_num > 10:
    c_num = 0

print(c_num)


if c_num !=0:
    met_id = met.id[0:-3]+'('+carbon_label[0:c_num]+')'
else:
    met_id = met.id[0:-3]

print(met_id)
print(len(met.formula))

In [ ]:
one_carbon_list = ['methanopterin', 'h4mpt', 'methanofuran', 'tetrahydrofolate']

if '_c0' in md.metabolites.cpd00087_c0.name:
    string = md.metabolites.cpd00087_c0.name[:-3]
elif '_e0' in md.metabolites.cpd00087_c0.name:
    string = md.metabolites.cpd00087_c0.name[:-3]
else:
    string = md.metabolites.cpd00087_c0.name

if any(substring in string.lower() for substring in substring_list):
    print('yes')
else:
    print('no')

In [ ]:
if '_e0' or '_c0' in md.metabolites.cpd01024_c0.name:
    print('yes')
else:
    print('no')

In [ ]:
'_e0' or '_c0' in md.metabolites.cpd01024_c0.name

In [ ]:
a = []
for met, coeff in md.reactions.get_by_id(t1).metabolites.items():
    a.append(str(abs(coeff))+' '+met.id)
    
a = ' + '.join(a)
a

In [ ]:
for met, coeff in md.reactions.get_by_id(t1).metabolites.items():
    if coeff == -1:
        print(met)

In [ ]:
t1 = md.reactions.get_by_id('rxn07847_c0')
for met, coeff in t1.metabolites.items():
    print(met.formula, coeff)

In [ ]:
m1 = md.metabolites.get_by_id('cpd00054_c0')
m1.formula
for i in m1.formula:
    print(i)

In [ ]:
### Create a list of ff Reactions

ff_md = ff.Model('sMMO_ff_model')
rxn_list = []

for rxn in md.reactions:

    if rxn.lower_bound * rxn.upper_bound < 0:
        ff_rxn = ff.Reaction(id=rxn.id, reversible = True)
    else:
        ff_rxn = ff.Reaction(id=rxn.id, reversible = False)


    for met,coeff in rxn.metabolites.items():
        if coeff < 0:
            ff_rxn.add_substrates(substrates = met_dict[met], stoichiometry = abs(coeff))
        elif coeff > 0:
            ff_rxn.add_products(products= met_dict[met], stoichiometry = abs(coeff))

    rxn_list.append(ff_rxn)
    ff_md.add_reactions(reactions=ff_rxn)
    # print(ff_rxn)

### Five metabolites have no reactions:
### cpd00074_c0, cpd00649_c0, cpd00792_c0, cpd11609_c0, cpd11610_c0
print(len(md.metabolites), ff_md.n_metabolites)
print(len(md.reactions), ff_md.n_reactions)

# for met in md.metabolites:
#     if met.id not in ff_md.metabolites:
#         print(met.id)


In [ ]:
with ff_md.optimizer() as opt:

    for rxn in list(ff_md.reactions_info.keys()):
        rxn_lb = md.reactions.get_by_id(rxn).lower_bound
        rxn_ub = md.reactions.get_by_id(rxn).upper_bound
        opt.set_flux_bounds(fluxid = rxn, bounds = [rxn_lb, rxn_ub])
    # opt.prepare()
    res = opt.optimize(objective = {'bio1_biomass': 1}, direction='max')

print('objective:', res.opt_objective)
        

In [ ]:
print('objective:', res.opt_objective)
print(res.opt_fluxes['EX_cpd00007_e0'])
print(res.opt_fluxes['EX_cpd01024_e0'])

In [ ]:
type(ff_md.reactions_info)

In [ ]:
MODEL_FILE = 'freeflux/models/toy/reactions.tsv' 
MEASURED_MDVS = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/test_MDVs.xlsx'
MEASURED_FLUXES = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/test_fluxes.xlsx'
OUT_DIR = '/Users/junwon/Library/CloudStorage/OneDrive-Umich/GEM/MFA13C/results/'

In [ ]:
def test_steady_state_fitting():

    # ecoli = Model('ecoli')
    # ecoli.read_from_file(MODEL_FILE)

    ecoli = ff_md
    
    with ecoli.fitter('ss') as fit:
        # specify the lableing strategy, 
        # use this method for every labeled substrate
        fit.set_labeling_strategy(
            'cpd01024_e0', 
            labeling_pattern = ['1'], 
            percentage = [1], 
            purity = [0.99]
        )
        
        # read measurements
        fit.set_measured_MDVs_from_file(MEASURED_MDVS)
        fit.set_measured_fluxes_from_file(MEASURED_FLUXES)
        
        # # set upper and lower bounds for fluxes
        # fit.set_flux_bounds('all', bounds = [-100, 100]) 

        # solve the fluxes
        fit.prepare(n_jobs = 3)
        while True:
            res = fit.solve(solver = 'ralg')
            if res.optimization_successful:
                break
    
    # save the results
    pd.Series(res.opt_net_fluxes).to_excel(
        OUT_DIR+'/estimated_net_fluxes.xlsx'
    )
    pd.Series(res.opt_total_fluxes).to_excel(
        OUT_DIR+'/estimated_total_fluxes.xlsx'
    )

    net_cis = res.estimate_confidence_intervals(
        which = 'net', 
        confidence_level = 0.95
    )
    pd.DataFrame(net_cis, index = ['LB', 'UB']).T.to_excel(
        OUT_DIR+'/netflux_le_CIs.xlsx'
    )
    
    total_cis = res.estimate_confidence_intervals(
        which = 'total', 
        confidence_level = 0.95
    )
    pd.DataFrame(total_cis, index = ['LB', 'UB']).T.to_excel(
        OUT_DIR+'/totalflux_le_CIs.xlsx'
    )

    # normal probability plot of residuals
    res.plot_normal_probability(show_fig = False, output_dir = OUT_DIR)
    
    # compare simulations and measurements
    res.plot_simulated_vs_measured_MDVs(show_fig = False, output_dir = OUT_DIR)
    res.plot_simulated_vs_measured_fluxes(show_fig = False, output_dir = OUT_DIR)
    
    # export the contribution matrix
    res.estimate_contribution_matrix(which = 'net').to_excel(
        OUT_DIR+'/netflux_contribMat.xlsx'
    )
    res.estimate_contribution_matrix(which = 'total').to_excel(
        OUT_DIR+'/totalflux_contribMat.xlsx'
    )
    
    # export the sensitivity matrix
    res.estimate_sensitivity(which = 'net').to_excel(
        OUT_DIR+'/netflux_senMat.xlsx'
    )
    res.estimate_sensitivity(which = 'total').to_excel(
        OUT_DIR+'/totalflux_senMat.xlsx'
    )



In [ ]:
test_steady_state_fitting()

In [ ]:
rxn = md.reactions.get_by_id('EX_cpd00007_e0')

if rxn.lower_bound * rxn.upper_bound < 0:
    ff_rxn = ff.Reaction(id=rxn.id, reversible = True)
else:
    ff_rxn = ff.Reaction(id=rxn.id, reversible = False)


for met,coeff in rxn.metabolites.items():
    if coeff < 0:
        ff_rxn.add_substrates(substrates = met_dict[met], stoichiometry = abs(coeff))
    elif coeff > 0:
        ff_rxn.add_products(products= met_dict[met], stoichiometry = abs(coeff))

print(ff_rxn)

In [ ]:
md.reactions.EX_cpd00007_e0

In [ ]:
md.reactions.rxn04996_c0

In [ ]:
md.metabolites.cpd00001_e0

In [ ]:
md.reactions.EX_cpd00001_e0.summary(solution = md_sol)

In [ ]:
md.objective = 'bio1_biomass'
md_sol = md.optimize()
md.summary(md_sol)

In [ ]:
md.metabolites.cpd00649_c0

In [ ]:
for a, b in md.reactions.rxn00224_c0.metabolites.items():
    if a in met_dict.keys():
        print(met_dict[a])

In [ ]:
for a, b in md.reactions.rxn00224_c0.metabolites.items():
    print(a)

In [ ]:
a in met_dict.keys()

In [ ]:
list(met_dict.keys())